# HEC-RAS 2D HDF Data Analysis Notebook

This notebook demonstrates how to manipulate and analyze HEC-RAS 2D HDF data using the ras-commander library. It leverages the HdfBase, HdfUtils, HdfStruc, HdfMesh, HdfXsec, HdfBndry, HdfPlan, HdfResultsPlan, HdfResultsMesh, and HdfResultsXsec classes to streamline data extraction, processing, and visualization.


In [1]:
# Install ras-commander from pip (uncomment to install if needed)
#!pip install ras-commander
# This installs ras-commander and all dependencies

# Use this setting to disable plot generation within the notebook
generate_plots = True
# Use this setting to disable map generation within the notebook
generate_maps = True
# Set both to false for llm-friendly outputs

In [2]:
# Import all required modules
#from ras_commander import *  # Import all ras-commander modules

# Import the required libraries for this notebook
import h5py
import numpy as np
import pandas as pd
import requests
from tqdm import tqdm
import scipy
import xarray as xr
import geopandas as gpd
import matplotlib.pyplot as plt
from IPython import display
import psutil  # For getting system CPU info
from concurrent.futures import ThreadPoolExecutor, as_completed
import time
import subprocess
import sys
import os
import shutil
from datetime import datetime, timedelta
from pathlib import Path  # Ensure pathlib is imported for file operations
import rasterio
from rasterio.plot import show
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import matplotlib.patches as patches
from matplotlib.patches import ConnectionPatch
import logging


In [3]:
# This cell will try to import the pip package, if it fails it will 
# add the parent directory to the Python path and try to import again
# This assumes you are working in a subfolder of the ras-commander repository
# This allows a user's revisions to be tested locally without installing the package

import sys
from pathlib import Path

# Flexible imports to allow for development without installation 
#  ** Use this version with Jupyter Notebooks **
try:
    # Try to import from the installed package
    from ras_commander import *
except ImportError:
    # If the import fails, add the parent directory to the Python path
    import os
    current_file = Path(os.getcwd()).resolve()
    rascmdr_directory = current_file.parent
    sys.path.append(str(rascmdr_directory))
    print("Loading ras-commander from local dev copy")
    # Now try to import again
    from ras_commander import *
print("ras_commander imported successfully")


ras_commander imported successfully


# Use Example Project or Load Your Own Project

In [4]:
# To Use the HEC Example Project:
# Download the BaldEagleCrkMulti2D project from HEC and Run Plan 06

# Define the path to the BaldEagleCrkMulti2D project
current_dir = Path.cwd()  # Adjust if your notebook is in a different directory
the_path = current_dir / "example_projects" / "BaldEagleCrkMulti2D"
import logging

# Check if BaldEagleCrkMulti2D.p06.hdf exists (so we don't have to re-run the simulation when re-running or debugging)
hdf_file = the_path / "BaldEagleDamBrk.p06.hdf"

if not hdf_file.exists():
    # Initialize RasExamples and extract the BaldEagleCrkMulti2D project
    RasExamples.extract_project(["BaldEagleCrkMulti2D"])

    # Initialize the RAS project using the default global ras object
    init_ras_project(the_path, "6.6")
    logging.info(f"Bald Eagle project initialized with folder: {ras.project_folder}")
    
    logging.info(f"Bald Eagle object id: {id(ras)}")
    
    # Define the plan number to execute
    plan_number = "06"

    # Update run flags for the project
    RasPlan.update_run_flags(
        plan_number,
        geometry_preprocessor=True,
        unsteady_flow_simulation=True,
        run_sediment=False,
        post_processor=True,
        floodplain_mapping=False
    )

    # Execute Plan 06 using RasCmdr for Bald Eagle
    print(f"Executing Plan {plan_number} for the Bald Eagle Creek project...")
    success_the = RasCmdr.compute_plan(plan_number)
    if success_the:
        print(f"Plan {plan_number} executed successfully for Bald Eagle.\n")
    else:
        print(f"Plan {plan_number} execution failed for Bald Eagle.\n")
else:
    print("Project already exists. Skipping project extraction and plan execution.")
    # Initialize the RAS project using the default global ras object
    init_ras_project(the_path, "6.6")
    plan_number = "06"

2025-03-19 17:39:57 - ras_commander.RasPrj - INFO - Initializing global 'ras' object via init_ras_project function.
2025-03-19 17:39:57 - ras_commander.RasPrj - INFO - Project initialized. ras_object project folder: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D


Project already exists. Skipping project extraction and plan execution.


###  OPTIONAL: Use your own project instead

your_project_path = Path(r"D:\yourprojectpath")

init_ras_project(your_project_path, "6.6")
plan_number = "01"  # Plan number to use for this notebook 



### If you use this code cell, don't run the previous cell or change to markdown
### NOTE: Ensure the HDF Results file was generated by HEC-RAS Version 6.x or above

-----

# Explore Project Dataframes using 'ras' Object

In [5]:
# Show ras object info
ras.plan_df

['   plan_number unsteady_number geometry_number  \\\n', '0           13              07              06   \n', '1           15              12              08   \n', '2           17              09              10   \n', '3           18              10              11   \n', '4           19              11              12   \n', '5           03              13              09   \n', '6           04              01              13   \n', '7           02              01              01   \n', '8           01              01              01   \n', '9           05              02              03   \n', '10          06              03              09   \n', '\n', '                                 Plan Title Program Version  \\\n', '0                   PMF with Multi 2D Areas            5.10   \n', '1               1d-2D Dambreak Refined Grid            5.10   \n', '2                           2D to 1D No Dam            5.00   \n', '3                              2D to 2D Run            5.0
...
[Output truncated, 7410 characters total]

In [6]:
ras.unsteady_df

['  unsteady_number                                          full_path  \\\n', '0              07  c:\\GH\\ras-commander\\examples\\example_projects\\...   \n', '1              08  c:\\GH\\ras-commander\\examples\\example_projects\\...   \n', '2              09  c:\\GH\\ras-commander\\examples\\example_projects\\...   \n', '3              10  c:\\GH\\ras-commander\\examples\\example_projects\\...   \n', '4              11  c:\\GH\\ras-commander\\examples\\example_projects\\...   \n', '5              12  c:\\GH\\ras-commander\\examples\\example_projects\\...   \n', '6              13  c:\\GH\\ras-commander\\examples\\example_projects\\...   \n', '7              01  c:\\GH\\ras-commander\\examples\\example_projects\\...   \n', '8              02  c:\\GH\\ras-commander\\examples\\example_projects\\...   \n', '9              03  c:\\GH\\ras-commander\\examples\\example_projects\\...   \n', '\n', '  geometry_number                            Flow Title Program Version  \\\n', '0            
...
[Output truncated, 4443 characters total]

In [7]:
ras.boundaries_df 

['   unsteady_number  boundary_condition_number river_reach_name river_station  \\\n', '0               07                          1   Bald Eagle Cr.    Lock Haven   \n', '1               07                          2   Bald Eagle Cr.    Lock Haven   \n', '2               07                          3   Bald Eagle Cr.    Lock Haven   \n', '3               07                          4   Bald Eagle Cr.    Lock Haven   \n', '4               07                          5   Bald Eagle Cr.    Lock Haven   \n', '5               07                          6   Bald Eagle Cr.    Lock Haven   \n', '6               07                          7   Bald Eagle Cr.    Lock Haven   \n', '7               07                          8   Bald Eagle Cr.    Lock Haven   \n', '8               07                          9   Bald Eagle Cr.    Lock Haven   \n', '9               07                         10   Bald Eagle Cr.    Lock Haven   \n', '10              08                          1   Bald Eagle Cr.
...
[Output truncated, 29090 characters total]

In [8]:
ras.get_hdf_entries()

['   plan_number unsteady_number geometry_number                     Plan Title  \\\n', '10          06              03              09  Gridded Precip - Infiltration   \n', '\n', '   Program Version          Short Identifier                Simulation Date  \\\n', '10            6.00  Grid Precip Infiltration  09SEP2018,0000,14SEP2018,0000   \n', '\n', '   Computation Interval Mapping Interval Run HTab  ... DSS File  \\\n', '10                20SEC            10MIN        1  ...      dss   \n', '\n', '   Friction Slope Method UNET D2 SolverType UNET D2 Name  \\\n', '10                     1   Pardiso (Direct)  BaldEagleCr   \n', '\n', '                                     HDF_Results_Path Geom File  \\\n', '10  c:\\GH\\ras-commander\\examples\\example_projects\\...        09   \n', '\n', '                                            Geom Path  Flow File  \\\n', '10  c:\\GH\\ras-commander\\examples\\example_projects\\...         03   \n', '\n', '                                            Flow Path  \\\n', '10  c:\\GH\\ras-commander\\examples\\example_projects\\...   \n', '\n', '                                            full_path  \n', '10  c:\\GH\\ras-commander\\examples\\example_projects\\...  \n', '\n', '[1 rows x 29 columns]']

# Find Paths for Results and Geometry HDF's

In [9]:
# Get the plan HDF path for the plan_number defined above
plan_hdf_path = ras.plan_df.loc[ras.plan_df['plan_number'] == plan_number, 'HDF_Results_Path'].values[0]

In [10]:
plan_hdf_path

["'c:\\\\GH\\\\ras-commander\\\\examples\\\\example_projects\\\\BaldEagleCrkMulti2D\\\\BaldEagleDamBrk.p06.hdf'"]

In [11]:
# Alternate: Get the geometry HDF path if you are extracting geometry elements from the geometry HDF
geom_hdf_path = ras.plan_df.loc[ras.plan_df['plan_number'] == plan_number, 'Geom Path'].values[0] + '.hdf'

In [12]:
geom_hdf_path

["'c:\\\\GH\\\\ras-commander\\\\examples\\\\example_projects\\\\BaldEagleCrkMulti2D\\\\BaldEagleDamBrk.g09.hdf'"]

## RAS-Commander's Decorators Allow for Flexible Function Calling
You can call most of the functions in the HDF* Classes using any of the following:
1. Plan/Geometry Number (with or without leading zeros):
   - "01", "1" - Plan/geometry number as string
   - 1 - Plan/geometry number as integer
   - "p01", "p1" - Plan number with 'p' prefix
2. Direct File Paths:
   - pathlib.Path object pointing to HDF file
   - String path to HDF file

3. h5py.File Objects:
   - Already opened HDF file object

The @standardize_input decorator handles all these input types consistently:
   - Validates the input exists and is accessible
   - Converts to proper pathlib.Path object
   - Handles RAS object references
   - Provides logging and error handling

This flexibility makes it easier to work with HDF files in different contexts while maintaining consistent behavior 
across the codebase, and helps prevent strict typing from introducing unnecessary friction for LLM Coding.


-----

# 2D HDF Data Extraction Examples

In [13]:
# Extract runtime and compute time data as dataframe
print("\nExtracting runtime and compute time data")
runtime_df = HdfResultsPlan.get_runtime_data(hdf_path=plan_number)

2025-03-19 17:39:57 - ras_commander.HdfResultsPlan - INFO - Final validated HDF file path: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.p06.hdf
2025-03-19 17:39:57 - ras_commander.HdfResultsPlan - INFO - Extracting Plan Information from: BaldEagleDamBrk.p06.hdf
2025-03-19 17:39:57 - ras_commander.HdfResultsPlan - INFO - Plan Name: Gridded Precip - Infiltration
2025-03-19 17:39:57 - ras_commander.HdfResultsPlan - INFO - Simulation Duration (hours): 120.0



Extracting runtime and compute time data


In [14]:
runtime_df

['                       Plan Name                File Name  \\\n', '0  Gridded Precip - Infiltration  BaldEagleDamBrk.p06.hdf   \n', '\n', '  Simulation Start Time Simulation End Time  Simulation Duration (s)  \\\n', '0            2018-09-09          2018-09-14                 432000.0   \n', '\n', '   Simulation Time (hr) Completing Geometry (hr)  Preprocessing Geometry (hr)  \\\n', '0                 120.0                      N/A                     0.000026   \n', '\n', '  Completing Event Conditions (hr)  Unsteady Flow Computations (hr)  \\\n', '0                              N/A                          0.07388   \n', '\n', '   Complete Process (hr)  Unsteady Flow Speed (hr/hr)  \\\n', '0               0.080174                  1624.249443   \n', '\n', '   Complete Process Speed (hr/hr)  \n', '0                     1496.751841  ']

In [15]:
# For all of the RasGeomHdf Class Functions, we will use geom_hdf_path
print(geom_hdf_path)

# For the example project, plan 06 is associated with geometry 09
# If you want to call the geometry by number, call RasHdfGeom functions with a number
# Otherwise, if you want to look up geometry hdf path by plan number, follow the logic in the previous code cells

c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.g09.hdf


In [16]:
# Use HdfUtils for extracting projection
print("\nExtracting Projection from HDF")
projection = HdfBase.get_projection(hdf_path=geom_hdf_path)


2025-03-19 17:39:57 - ras_commander.HdfBase - INFO - Using HDF file from direct string path: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.g09.hdf
2025-03-19 17:39:57 - ras_commander.HdfBase - INFO - Final validated HDF file path: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.g09.hdf
2025-03-19 17:39:57 - ras_commander.HdfBase - INFO - Found projection in HDF file: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.g09.hdf



Extracting Projection from HDF


In [17]:
projection

['\'PROJCS["NAD_1983_StatePlane_Pennsylvania_North_FIPS_3701_Feet",GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Lambert_Conformal_Conic"],PARAMETER["False_Easting",1968500.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",-77.75],PARAMETER["Standard_Parallel_1",40.88333333333333],PARAMETER["Standard_Parallel_2",41.95],PARAMETER["Latitude_Of_Origin",40.16666666666666],UNIT["Foot_US",0.3048006096012192]]\'']

In [18]:
# Use HdfPlan for geometry-related operations
print("\nExtracting Geometry Information")
geom_attrs = HdfPlan.get_geometry_information(geom_hdf_path)

2025-03-19 17:39:57 - ras_commander.HdfPlan - INFO - Using HDF file from direct string path: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.g09.hdf
2025-03-19 17:39:57 - ras_commander.HdfPlan - INFO - Final validated HDF file path: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.g09.hdf



Extracting Geometry Information
Getting geometry attributes from c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.g09.hdf
Checking for Geometry group in c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.g09.hdf
Getting root level geometry attributes
Successfully extracted root level geometry attributes


In [19]:
geom_attrs

['                                                                                   Value\n', 'Complete Geometry                                                                   True\n', 'Extents                                [1960041.35636708, 2092643.59732271, 285497.89...\n', 'Geometry Time                                                        2025-03-19 17:32:59\n', 'Infiltration Date Last Modified                                      2022-03-11 13:52:44\n', 'Infiltration File Date                                               2020-11-24 13:24:58\n', 'Infiltration Filename                                      .\\Soils Data\\Infiltration.hdf\n', 'Infiltration Layername                                                      Infiltration\n', 'Land Cover Date Last Modified                                        2022-03-11 13:45:08\n', 'Land Cover File Date                                                 2022-03-11 13:45:08\n', 'Land Cover Filename                                  .\\
...
[Output truncated, 2074 characters total]

In [20]:
# Use HdfMesh for geometry-related operations
print("\nListing 2D Flow Area Names")
flow_area_names = HdfMesh.get_mesh_area_names(geom_hdf_path)

2025-03-19 17:39:57 - ras_commander.HdfMesh - INFO - Using HDF file from direct string path: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.g09.hdf
2025-03-19 17:39:57 - ras_commander.HdfMesh - INFO - Final validated HDF file path: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.g09.hdf



Listing 2D Flow Area Names


In [21]:
print("2D Flow Area Name (returned as list):")
flow_area_names
# Note: this is returned as a list because it is used internally by other functions.  

2D Flow Area Name (returned as list):


["['BaldEagleCr']"]

In [22]:
# Get 2D Flow Area Attributes (get_mesh_area_attributes)
print("\nExtracting 2D Flow Area Attributes")
flow_area_attributes = HdfMesh.get_mesh_area_attributes(geom_hdf_path)

2025-03-19 17:39:57 - ras_commander.HdfMesh - INFO - Using HDF file from direct string path: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.g09.hdf
2025-03-19 17:39:57 - ras_commander.HdfMesh - INFO - Final validated HDF file path: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.g09.hdf



Extracting 2D Flow Area Attributes


In [23]:
flow_area_attributes

['                                 Value\n', "Name                    b'BaldEagleCr'\n", 'Locked                               0\n', 'Mann                              0.04\n', 'Multiple Face Mann n                 0\n', 'Composite LC                         0\n', 'Cell Vol Tol                      0.01\n', 'Cell Min Area Fraction            0.01\n', 'Face Profile Tol                  0.01\n', 'Face Area Tol                     0.01\n', 'Face Conv Ratio                   0.02\n', 'Laminar Depth                      0.2\n', 'Min Face Length Ratio             0.05\n', 'Spacing dx                       250.0\n', 'Spacing dy                       250.0\n', 'Shift dx                           NaN\n', 'Shift dy                           NaN\n', 'Cell Count                       18066']

In [24]:
# Get 2D Flow Area Perimeter Polygons (get_mesh_areas)
print("\nExtracting 2D Flow Area Perimeter Polygons")
mesh_areas = HdfMesh.get_mesh_areas(geom_hdf_path)

2025-03-19 17:39:57 - ras_commander.HdfMesh - INFO - Using HDF file from direct string path: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.g09.hdf
2025-03-19 17:39:57 - ras_commander.HdfMesh - INFO - Final validated HDF file path: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.g09.hdf
2025-03-19 17:39:57 - ras_commander.HdfMesh - INFO - Using existing Path object HDF file: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.g09.hdf
2025-03-19 17:39:57 - ras_commander.HdfMesh - INFO - Final validated HDF file path: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.g09.hdf
2025-03-19 17:39:57 - ras_commander.HdfBase - INFO - Using HDF file from h5py.File object: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.g09.hdf
2025-03-19 17:39:57 - ras_commander.HdfBase - INFO - Final validated HDF file path: c:\GH\ras-commander\exam


Extracting 2D Flow Area Perimeter Polygons


In [25]:
mesh_areas

['     mesh_name                                           geometry\n', '0  BaldEagleCr  POLYGON ((2009315.708 321138.385, 2009371.858 ...']

In [26]:
# Generate Map of Mesh Areas
if generate_plots:
    # Plot the 2D Flow Area Perimeter Polygons
    import matplotlib.pyplot as plt

    fig, ax = plt.subplots(figsize=(12, 8))
    mesh_areas.plot(ax=ax, edgecolor='black', facecolor='none')

    # Add labels for each polygon
    for idx, row in mesh_areas.iterrows():
        centroid = row.geometry.centroid
        # Check if 'Name' column exists, otherwise use a default label
        label = row.get('Name', f'Area {idx}')
        ax.annotate(label, (centroid.x, centroid.y), ha='center', va='center')

    plt.title('2D Flow Area Perimeter Polygons')
    plt.xlabel('Easting')
    plt.ylabel('Northing')
    plt.tight_layout()
    plt.show()

<Figure size 1200x800 with 1 Axes>

In [27]:
# Get mesh cell faces as geodatframe
mesh_cell_faces_gdf = HdfMesh.get_mesh_cell_faces(geom_hdf_path)


2025-03-19 17:39:58 - ras_commander.HdfMesh - INFO - Using HDF file from direct string path: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.g09.hdf
2025-03-19 17:39:58 - ras_commander.HdfMesh - INFO - Final validated HDF file path: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.g09.hdf
2025-03-19 17:39:58 - ras_commander.HdfMesh - INFO - Using existing Path object HDF file: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.g09.hdf
2025-03-19 17:39:58 - ras_commander.HdfMesh - INFO - Final validated HDF file path: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.g09.hdf
2025-03-19 17:39:58 - ras_commander.HdfBase - INFO - Using HDF file from h5py.File object: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.g09.hdf
2025-03-19 17:39:58 - ras_commander.HdfBase - INFO - Final validated HDF file path: c:\GH\ras-commander\exam

In [28]:
mesh_cell_faces_gdf

['         mesh_name  face_id                                           geometry\n', '0      BaldEagleCr        0        LINESTRING (2042125 351625, 2042375 351625)\n', '1      BaldEagleCr        1        LINESTRING (2042375 351625, 2042375 351875)\n', '2      BaldEagleCr        2        LINESTRING (2042375 351875, 2042125 351875)\n', '3      BaldEagleCr        3        LINESTRING (2042125 351875, 2042125 351625)\n', '4      BaldEagleCr        4        LINESTRING (2042375 351375, 2042375 351625)\n', '...            ...      ...                                                ...\n', '37589  BaldEagleCr    37589  LINESTRING (1978444.054 300812.794, 1978402.01...\n', '37590  BaldEagleCr    37590  LINESTRING (1973531.76 297370.846, 1973246.989...\n', '37591  BaldEagleCr    37591  LINESTRING (1968886.801 295900.5, 1968782.778 ...\n', '37592  BaldEagleCr    37592  LINESTRING (1965981.775 291978.68, 1966280.11 ...\n', '37593  BaldEagleCr    37593  LINESTRING (1969588.438 289578.114, 1969731.65...\n', '\n', '[37594 rows x 3 columns]']

In [29]:
from matplotlib.collections import LineCollection
import numpy as np

# Calculate and display statistics
print("\nMesh Cell Faces Statistics:")
print(f"Total number of cell faces: {len(mesh_cell_faces_gdf)}")
print(f"Number of unique meshes: {mesh_cell_faces_gdf['mesh_name'].nunique()}")

if generate_maps:
    # Plot the mesh cell faces more efficiently
    fig, ax = plt.subplots(figsize=(12, 8))

    # Convert all geometries to numpy arrays at once for faster plotting
    lines = [list(zip(*line.xy)) for line in mesh_cell_faces_gdf.geometry]
    lines_collection = LineCollection(lines, colors='blue', linewidth=0.5, alpha=0.5)
    ax.add_collection(lines_collection)

    # Set plot title and labels
    plt.title('Mesh Cell Faces')
    plt.xlabel('Easting')
    plt.ylabel('Northing')

    # Calculate centroids once and store as numpy arrays
    centroids = np.array([[geom.centroid.x, geom.centroid.y] for geom in mesh_cell_faces_gdf.geometry])

    # Create scatter plot with numpy arrays
    scatter = ax.scatter(
        centroids[:, 0],
        centroids[:, 1], 
        c=mesh_cell_faces_gdf['face_id'],
        cmap='viridis',
        s=1,
        alpha=0.5
    )
    plt.colorbar(scatter, label='Face ID')

    # Set axis limits based on data bounds
    ax.set_xlim(centroids[:, 0].min(), centroids[:, 0].max())
    ax.set_ylim(centroids[:, 1].min(), centroids[:, 1].max())

    plt.tight_layout()
    plt.show()
else:
    print("generate_maps is False")



Mesh Cell Faces Statistics:
Total number of cell faces: 37594
Number of unique meshes: 1


<Figure size 1200x800 with 2 Axes>

In [30]:
# Function to find the nearest cell face to a given point
def find_nearest_cell_face(point, cell_faces_df):
    """
    Find the nearest cell face to a given point.

    Args:
        point (shapely.geometry.Point): The input point.
        cell_faces_df (GeoDataFrame): DataFrame containing cell face linestrings.

    Returns:
        int: The face_id of the nearest cell face.
        float: The distance to the nearest cell face.
    """
    # Calculate distances from the input point to all cell faces
    distances = cell_faces_df.geometry.distance(point)

    # Find the index of the minimum distance
    nearest_index = distances.idxmin()

    # Get the face_id and distance of the nearest cell face
    nearest_face_id = cell_faces_df.loc[nearest_index, 'face_id']
    nearest_distance = distances[nearest_index]

    return nearest_face_id, nearest_distance

# Example usage
print("\nFinding the nearest cell face to a given point")

# Create a sample point (you can replace this with any point of interest)
from shapely.geometry import Point
from geopandas import GeoDataFrame

# Get the centroid of the mesh cell faces
print("Getting Centroid of 2D Mesh Polygon")
centroid = mesh_cell_faces_gdf.geometry.union_all().centroid

# Create GeoDataFrame with the centroid point, using same CRS as mesh_cell_faces_gdf
sample_point = GeoDataFrame(
    {'geometry': [centroid]}, 
    crs=mesh_cell_faces_gdf.crs
)

if not mesh_cell_faces_gdf.empty and not sample_point.empty:
    print("Searching Cell")
    nearest_face_id, distance = find_nearest_cell_face(sample_point.geometry.iloc[0], mesh_cell_faces_gdf)
    print(f"Nearest cell face to point {sample_point.geometry.iloc[0].coords[0]}:")
    print(f"Face ID: {nearest_face_id}")
    print(f"Distance: {distance:.2f} units")


Finding the nearest cell face to a given point
Getting Centroid of 2D Mesh Polygon
Searching Cell
Nearest cell face to point (2030989.2589121135, 336969.5028173043):
Face ID: 19961
Distance: 94.50 units


In [31]:
# Generate map of cell faces with sample point and nearest cell face shown
if generate_maps:
    # Visualize the result
    fig, ax = plt.subplots(figsize=(12, 8))
    
    # Plot all cell faces
    mesh_cell_faces_gdf.plot(ax=ax, color='blue', linewidth=0.5, alpha=0.5, label='Cell Faces')
    
    # Plot the sample point
    sample_point.plot(ax=ax, color='red', markersize=100, alpha=0.7, label='Sample Point')
    
    # Plot the nearest cell face
    nearest_face = mesh_cell_faces_gdf[mesh_cell_faces_gdf['face_id'] == nearest_face_id]
    nearest_face.plot(ax=ax, color='green', linewidth=2, alpha=0.7, label='Nearest Face')
    
    # Set labels and title
    ax.set_xlabel('X Coordinate')
    ax.set_ylabel('Y Coordinate')
    ax.set_title('Nearest Cell Face to Sample Point')
    
    # Add legend and grid
    ax.legend()
    ax.grid(True)
    
    # Adjust layout and display
    plt.tight_layout()
    plt.show()
else:
    print("generate_maps is set to False")


<Figure size 1200x800 with 1 Axes>

In [32]:
# Extract Cell Polygons
print("\nExample 6: Extracting Cell Polygons")
cell_polygons_df = HdfMesh.get_mesh_cell_polygons(geom_hdf_path)

2025-03-19 17:40:04 - ras_commander.HdfMesh - INFO - Using HDF file from direct string path: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.g09.hdf
2025-03-19 17:40:04 - ras_commander.HdfMesh - INFO - Final validated HDF file path: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.g09.hdf
2025-03-19 17:40:04 - ras_commander.HdfMesh - INFO - Using existing Path object HDF file: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.g09.hdf
2025-03-19 17:40:04 - ras_commander.HdfMesh - INFO - Final validated HDF file path: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.g09.hdf
2025-03-19 17:40:04 - ras_commander.HdfMesh - INFO - Using existing Path object HDF file: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.g09.hdf
2025-03-19 17:40:04 - ras_commander.HdfMesh - INFO - Final validated HDF file path: c:\GH\ras-commander\examp


Example 6: Extracting Cell Polygons


2025-03-19 17:40:04 - ras_commander.HdfBase - INFO - Using HDF file from h5py.File object: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.g09.hdf
2025-03-19 17:40:04 - ras_commander.HdfBase - INFO - Final validated HDF file path: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.g09.hdf
2025-03-19 17:40:04 - ras_commander.HdfBase - INFO - Found projection in HDF file: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.g09.hdf
2025-03-19 17:40:06 - ras_commander.HdfBase - INFO - Using HDF file from h5py.File object: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.g09.hdf
2025-03-19 17:40:06 - ras_commander.HdfBase - INFO - Final validated HDF file path: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.g09.hdf
2025-03-19 17:40:06 - ras_commander.HdfBase - INFO - Found projection in HDF file: c:\GH\ras-commander\examples\examp

In [33]:
cell_polygons_df

['         mesh_name  cell_id                                           geometry\n', '0      BaldEagleCr        0  POLYGON ((2082875 370625, 2082723.922 370776.0...\n', '1      BaldEagleCr        1  POLYGON ((2083125 370625, 2083125 370844.185, ...\n', '2      BaldEagleCr        2  POLYGON ((2083375 370625, 2083375 370886.638, ...\n', '3      BaldEagleCr        3  POLYGON ((2083625 370625, 2083625 370925.693, ...\n', '4      BaldEagleCr        4  POLYGON ((2083875 370625, 2083875 370958.588, ...\n', '...            ...      ...                                                ...\n', '18061  BaldEagleCr    18061  POLYGON ((2060605.501 351955.442, 2060652.06 3...\n', '18062  BaldEagleCr    18062  POLYGON ((2060652.06 351777.237, 2060849.045 3...\n', '18063  BaldEagleCr    18063  POLYGON ((2060875 351951.343, 2060929.66 35193...\n', '18064  BaldEagleCr    18064  POLYGON ((2060991.954 351624.893, 2060964.72 3...\n', '18065  BaldEagleCr    18065  POLYGON ((2061172.274 351856.061, 2061298.822 ...\n', '\n', '[18066 rows x 3 columns]']

In [34]:
# Plot cell polygons

if generate_maps:
    fig, ax = plt.subplots(figsize=(12, 8))

    # Plot cell polygons
    cell_polygons_df.plot(ax=ax, edgecolor='blue', facecolor='none')

    # Set labels and title
    ax.set_xlabel('X Coordinate')
    ax.set_ylabel('Y Coordinate')
    ax.set_title('2D Flow Area Cell Polygons')

    # Add grid
    ax.grid(True)

    # Adjust layout and display
    plt.tight_layout()
    plt.show()
else:
    print("generate_maps is set to False")


<Figure size 1200x800 with 1 Axes>

In [35]:
# Extract Cell Info
print("\nExample 5: Extracting Cell Info")
cell_info_df = HdfMesh.get_mesh_cell_points(geom_hdf_path)

2025-03-19 17:40:09 - ras_commander.HdfMesh - INFO - Using HDF file from direct string path: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.g09.hdf
2025-03-19 17:40:09 - ras_commander.HdfMesh - INFO - Final validated HDF file path: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.g09.hdf
2025-03-19 17:40:09 - ras_commander.HdfMesh - INFO - Using existing Path object HDF file: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.g09.hdf
2025-03-19 17:40:09 - ras_commander.HdfMesh - INFO - Final validated HDF file path: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.g09.hdf
2025-03-19 17:40:09 - ras_commander.HdfBase - INFO - Using HDF file from h5py.File object: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.g09.hdf
2025-03-19 17:40:09 - ras_commander.HdfBase - INFO - Final validated HDF file path: c:\GH\ras-commander\exam


Example 5: Extracting Cell Info


In [36]:
cell_info_df

['         mesh_name  cell_id                        geometry\n', '0      BaldEagleCr        0          POINT (2083000 370750)\n', '1      BaldEagleCr        1          POINT (2083250 370750)\n', '2      BaldEagleCr        2          POINT (2083500 370750)\n', '3      BaldEagleCr        3          POINT (2083750 370750)\n', '4      BaldEagleCr        4          POINT (2084000 370750)\n', '...            ...      ...                             ...\n', '19592  BaldEagleCr    19592  POINT (1978423.032 300718.897)\n', '19593  BaldEagleCr    19593  POINT (1973389.375 297311.928)\n', '19594  BaldEagleCr    19594   POINT (1968834.79 295808.861)\n', '19595  BaldEagleCr    19595  POINT (1966130.942 291879.395)\n', '19596  BaldEagleCr    19596   POINT (1969660.046 289673.23)\n', '\n', '[19597 rows x 3 columns]']

In [37]:
# Plot cell centers

if generate_maps:
    fig, ax = plt.subplots(figsize=(12, 8))

    # Plot cell centers
    cell_info_df.plot(ax=ax, color='red', markersize=5)

    # Set labels and title
    ax.set_xlabel('X Coordinate')
    ax.set_ylabel('Y Coordinate')
    ax.set_title('2D Flow Area Cell Centers')

    # Add grid
    ax.grid(True)

    # Adjust layout and display
    plt.tight_layout()
    plt.show()
else:
    print("generate_maps is set to False")



<Figure size 1200x800 with 1 Axes>

In [38]:
# Function to find the nearest cell center to a given point
def find_nearest_cell(point, cell_centers_df):
    """
    Find the nearest cell center to a given point.

    Args:
        point (shapely.geometry.Point): The input point.
        cell_centers_df (GeoDataFrame): DataFrame containing cell center points.

    Returns:
        int: The cell_id of the nearest cell.
        float: The distance to the nearest cell center.
    """
    # Calculate distances from the input point to all cell centers
    distances = cell_centers_df.geometry.distance(point)

    # Find the index of the minimum distance
    nearest_index = distances.idxmin()

    # Get the cell_id and distance of the nearest cell
    nearest_cell_id = cell_centers_df.loc[nearest_index, 'cell_id']
    nearest_distance = distances[nearest_index]

    return nearest_cell_id, nearest_distance

# Example usage
print("\nFinding the nearest cell to a given point")

# Sample point was created in a previous code cell 

# Get the projection from the geometry file
# projection = HdfUtils.get_projection(hdf_path=geom_hdf_path) # This was done in a previous code cell
if projection:
    print(f"Using projection: {projection}")
else:
    print("No projection information found. Using default CRS.")
    projection = "EPSG:4326"  # Default to WGS84 if no projection is found



# Ensure the CRS of the sample point matches the cell_info_df
if sample_point.crs != cell_info_df.crs:
    sample_point = sample_point.to_crs(cell_info_df.crs)

nearest_cell_id, distance = find_nearest_cell(sample_point.geometry.iloc[0], cell_info_df)
print(f"Nearest cell to point {sample_point.geometry.iloc[0].coords[0]}:")
print(f"Cell ID: {nearest_cell_id}")
print(f"Distance: {distance:.2f} units")

if generate_maps:
    # Visualize the result
    fig, ax = plt.subplots(figsize=(12, 8))

    # Plot all cell centers
    cell_info_df.plot(ax=ax, color='blue', markersize=5, alpha=0.5, label='Cell Centers')

    # Plot the sample point
    sample_point.plot(ax=ax, color='red', markersize=100, alpha=0.7, label='Sample Point')

    # Plot the nearest cell center
    nearest_cell = cell_info_df[cell_info_df['cell_id'] == nearest_cell_id]
    nearest_cell.plot(ax=ax, color='green', markersize=100, alpha=0.7, label='Nearest Cell')

    # Set labels and title
    ax.set_xlabel('X Coordinate')
    ax.set_ylabel('Y Coordinate')
    ax.set_title('Nearest Cell to Sample Point')

    # Add legend and grid
    ax.legend()
    ax.grid(True)

    # Adjust layout and display
    plt.tight_layout()
    plt.show()
else:
    print("generate_maps is set to False")




Finding the nearest cell to a given point
Using projection: PROJCS["NAD_1983_StatePlane_Pennsylvania_North_FIPS_3701_Feet",GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Lambert_Conformal_Conic"],PARAMETER["False_Easting",1968500.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",-77.75],PARAMETER["Standard_Parallel_1",40.88333333333333],PARAMETER["Standard_Parallel_2",41.95],PARAMETER["Latitude_Of_Origin",40.16666666666666],UNIT["Foot_US",0.3048006096012192]]
Nearest cell to point (2030989.2589121135, 336969.5028173043):
Cell ID: 9249
Distance: 32.33 units


<Figure size 1200x800 with 1 Axes>

In [39]:
# Get geometry structures attributes
print("\nGetting geometry structures attributes as Dataframe")
geom_structures_attrs = HdfStruc.get_geom_structures_attrs(geom_hdf_path)

2025-03-19 17:40:11 - ras_commander.HdfStruc - INFO - Using HDF file from direct string path: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.g09.hdf
2025-03-19 17:40:11 - ras_commander.HdfStruc - INFO - Final validated HDF file path: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.g09.hdf



Getting geometry structures attributes as Dataframe


In [40]:
geom_structures_attrs

['   Bridge/Culvert Count  Connection Count  Has Bridge Opening (2D)  \\\n', '0                     0                 1                        0   \n', '\n', '   Inline Structure Count  Lateral Structure Count  \n', '0                       0                        0  ']

In [41]:
# TODO: Paths and Functions for each type of structure: 

# Getting geometry structures attributes
# Geometry structures attributes:
# Bridge/Culvert Count: 0
# Connection Count: 4
# Has Bridge Opening (2D): 0
# Inline Structure Count: 0
# Lateral Structure Count: 0

In [42]:
# Get boundary condition lines
print("\nExtracting Boundary Condition Lines as Geodataframe")
bc_lines_df = HdfBndry.get_bc_lines(geom_hdf_path)

2025-03-19 17:40:11 - ras_commander.HdfBndry - INFO - Using HDF file from direct string path: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.g09.hdf
2025-03-19 17:40:11 - ras_commander.HdfBndry - INFO - Final validated HDF file path: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.g09.hdf
2025-03-19 17:40:11 - ras_commander.HdfBase - INFO - Using existing Path object HDF file: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.g09.hdf
2025-03-19 17:40:11 - ras_commander.HdfBase - INFO - Final validated HDF file path: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.g09.hdf
2025-03-19 17:40:11 - ras_commander.HdfBase - INFO - Using HDF file from h5py.File object: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.g09.hdf
2025-03-19 17:40:11 - ras_commander.HdfBase - INFO - Final validated HDF file path: c:\GH\ras-commander\ex


Extracting Boundary Condition Lines as Geodataframe


In [43]:
bc_lines_df

['              Name        SA-2D      Type       Length  \\\n', '0    DSNormalDepth  BaldEagleCr  External  1801.303345   \n', '1       DS2NormalD  BaldEagleCr  External  2633.932617   \n', '2  Upstream Inflow  BaldEagleCr  External  2533.638672   \n', '\n', '                                            geometry  bc_line_id  \n', '0  LINESTRING (2082004.235 364024.82, 2083193.546...           0  \n', '1  LINESTRING (2084425.804 365392.892, 2084354.64...           1  \n', '2  LINESTRING (1967473.737 290973.629, 1969582.89...           2  ']

In [44]:
# Plot Boundary Condition Lines with Perimeter

if generate_maps:
    fig, ax = plt.subplots(figsize=(12, 8))

    if not mesh_areas.empty:
        mesh_areas.plot(ax=ax, edgecolor='black', facecolor='none', alpha=0.7, label='2D Flow Area')
        
        # Add labels for each polygon
        for idx, row in mesh_areas.iterrows():
            centroid = row.geometry.centroid
            label = row.get('Name', f'Area {idx}')
            ax.annotate(label, (centroid.x, centroid.y), ha='center', va='center')

    # Plot boundary condition lines
    if not bc_lines_df.empty:
        bc_lines_df.plot(ax=ax, color='red', linewidth=2, label='Boundary Condition Lines')

    # Set labels and title
    ax.set_xlabel('Easting')
    ax.set_ylabel('Northing')
    ax.set_title('2D Flow Area Perimeter Polygons and Boundary Condition Lines')

    # Add grid and legend
    ax.grid(True)
    ax.legend()

    # Adjust layout and display
    plt.tight_layout()
    plt.show()

else:
    print("generate_maps is set to False")
# Plot 2D Flow Area Perimeter Polygons

C:\Users\billk\AppData\Local\Temp\ipykernel_2528\268397963.py:26: UserWarning: Legend does not support handles for PatchCollection instances.
See: https://matplotlib.org/stable/tutorials/intermediate/legend_guide.html#implementing-a-custom-legend-handler
  ax.legend()


<Figure size 1200x800 with 1 Axes>

In [45]:
# Extract Breaklines as Geodataframe
print("\nExtracting Breaklines")
breaklines_gdf = HdfBndry.get_breaklines(geom_hdf_path)


2025-03-19 17:40:11 - ras_commander.HdfBndry - INFO - Using HDF file from direct string path: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.g09.hdf
2025-03-19 17:40:11 - ras_commander.HdfBndry - INFO - Final validated HDF file path: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.g09.hdf
2025-03-19 17:40:11 - ras_commander.HdfBase - INFO - Using HDF file from h5py.File object: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.g09.hdf
2025-03-19 17:40:11 - ras_commander.HdfBase - INFO - Final validated HDF file path: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.g09.hdf
2025-03-19 17:40:11 - ras_commander.HdfBase - INFO - Found projection in HDF file: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.g09.hdf



Extracting Breaklines


In [46]:
breaklines_gdf

['   bl_id       Name                                           geometry\n', '0      0  SayersDam  LINESTRING (2002361.246 323707.927, 2002741.35...\n', '1      1      Lower  LINESTRING (2060356.422 351786.819, 2060316.47...\n', '2      2     Middle  LINESTRING (2052757.788 348470.547, 2052785.84...\n', '3      3      Upper  LINESTRING (2045597.199 348412.994, 2045638.91...']

In [47]:
# Plot breaklines and 2D Flow Area Perimeter Polygons

if generate_plots:
    fig, ax = plt.subplots(figsize=(12, 8))

    # Plot 2D Flow Area Perimeter Polygons
    if not mesh_areas.empty:
        mesh_areas.plot(ax=ax, edgecolor='black', facecolor='none', alpha=0.7, label='2D Flow Area')
        
        # Add labels for each polygon
        for idx, row in mesh_areas.iterrows():
            centroid = row.geometry.centroid
            label = row.get('Name', f'Area {idx}')
            ax.annotate(label, (centroid.x, centroid.y), ha='center', va='center')

    # Plot breaklines
    if not breaklines_gdf.empty:
        breaklines_gdf.plot(ax=ax, color='blue', linewidth=2, label='Breaklines')

    # Set labels and title
    ax.set_xlabel('Easting')
    ax.set_ylabel('Northing')
    ax.set_title('2D Flow Area Perimeter Polygons and Breaklines')

    # Add grid and legend
    ax.grid(True)
    ax.legend()

    # Adjust layout and display
    plt.tight_layout()
    plt.show()

C:\Users\billk\AppData\Local\Temp\ipykernel_2528\3295158439.py:27: UserWarning: Legend does not support handles for PatchCollection instances.
See: https://matplotlib.org/stable/tutorials/intermediate/legend_guide.html#implementing-a-custom-legend-handler
  ax.legend()


<Figure size 1200x800 with 1 Axes>

In [48]:
# Get structures as GeoDatframe
structures_gdf = HdfStruc.get_structures(geom_hdf_path)

2025-03-19 17:40:12 - ras_commander.HdfStruc - INFO - Using HDF file from direct string path: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.g09.hdf
2025-03-19 17:40:12 - ras_commander.HdfStruc - INFO - Final validated HDF file path: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.g09.hdf
2025-03-19 17:40:12 - ras_commander.HdfBase - INFO - Using existing Path object HDF file: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.g09.hdf
2025-03-19 17:40:12 - ras_commander.HdfBase - INFO - Final validated HDF file path: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.g09.hdf
2025-03-19 17:40:12 - ras_commander.HdfBase - INFO - Found projection in HDF file: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.g09.hdf
2025-03-19 17:40:12 - ras_commander.HdfStruc - INFO - Successfully extracted structures GeoDataFrame.
2025-03-19 1

In [49]:
structures_gdf

['         Type                Mode River Reach RS  Connection  \\\n', '0  Connection  Weir/Gate/Culverts                 Sayers Dam   \n', '\n', '                 Groupname US Type US River US Reach  ... US XS Mann (Count)  \\\n', '0  BaldEagleCr, Sayers Dam      2D                    ...                  0   \n', '\n', '  US BR Mann (Index) US BR Mann (Count) DS XS Mann (Index) DS XS Mann (Count)  \\\n', '0                  0                  0                  0                  0   \n', '\n', '  DS BR Mann (Index) DS BR Mann (Count) RC (Index) RC (Count)  \\\n', '0                  0                  0          0          0   \n', '\n', '                                        Profile_Data  \n', "0  [{'Station': 0.0, 'Elevation': 683.0}, {'Stati...  \n", '\n', '[1 rows x 162 columns]']

In [50]:
# Get boundary condition lines as GeoDatframe
bc_lines_gdf = HdfBndry.get_bc_lines(geom_hdf_path)
print("\nBoundary Condition Lines:")

2025-03-19 17:40:12 - ras_commander.HdfBndry - INFO - Using HDF file from direct string path: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.g09.hdf
2025-03-19 17:40:12 - ras_commander.HdfBndry - INFO - Final validated HDF file path: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.g09.hdf
2025-03-19 17:40:12 - ras_commander.HdfBase - INFO - Using existing Path object HDF file: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.g09.hdf
2025-03-19 17:40:12 - ras_commander.HdfBase - INFO - Final validated HDF file path: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.g09.hdf
2025-03-19 17:40:12 - ras_commander.HdfBase - INFO - Using HDF file from h5py.File object: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.g09.hdf
2025-03-19 17:40:12 - ras_commander.HdfBase - INFO - Final validated HDF file path: c:\GH\ras-commander\ex


Boundary Condition Lines:


In [51]:
bc_lines_gdf

['              Name        SA-2D      Type       Length  \\\n', '0    DSNormalDepth  BaldEagleCr  External  1801.303345   \n', '1       DS2NormalD  BaldEagleCr  External  2633.932617   \n', '2  Upstream Inflow  BaldEagleCr  External  2533.638672   \n', '\n', '                                            geometry  bc_line_id  \n', '0  LINESTRING (2082004.235 364024.82, 2083193.546...           0  \n', '1  LINESTRING (2084425.804 365392.892, 2084354.64...           1  \n', '2  LINESTRING (1967473.737 290973.629, 1969582.89...           2  ']

### Dev Note: Need to add function for Reference Lines

In [52]:
# Get reference points as Geodataframe
ref_points_gdf = HdfBndry.get_reference_points(geom_hdf_path)

2025-03-19 17:40:12 - ras_commander.HdfBndry - INFO - Using HDF file from direct string path: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.g09.hdf
2025-03-19 17:40:12 - ras_commander.HdfBndry - INFO - Final validated HDF file path: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.g09.hdf


In [53]:
print("\nReference Points:")
ref_points_gdf
# There are no reference points in this example project (for demonstration only)


Reference Points:


""
[... additional rows truncated ...]


In [54]:
# Extract Refinement Regions
refinement_regions_df = HdfBndry.get_refinement_regions(geom_hdf_path)

2025-03-19 17:40:12 - ras_commander.HdfBndry - INFO - Using HDF file from direct string path: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.g09.hdf
2025-03-19 17:40:12 - ras_commander.HdfBndry - INFO - Final validated HDF file path: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.g09.hdf


In [55]:
refinement_regions_df

""
[... additional rows truncated ...]


In [56]:
# Plot Refinement Regions

if not refinement_regions_df.empty:
    print("Refinement Regions DataFrame:")
    display(refinement_regions_df.head())
    
    # Plot refinement regions
    fig, ax = plt.subplots(figsize=(12, 8))
    refinement_regions_df.plot(ax=ax, column='CellSize', legend=True, 
                               legend_kwds={'label': 'Cell Size', 'orientation': 'horizontal'},
                               cmap='viridis')
    ax.set_title('2D Mesh Area Refinement Regions')
    ax.set_xlabel('Easting')
    ax.set_ylabel('Northing')
    plt.tight_layout()
    plt.show()
else:
    print("No refinement regions found in the geometry file.")

# Analyze Refinement Regions
if not refinement_regions_df.empty:
    print("\nRefinement Regions Analysis:")
    print(f"Total number of refinement regions: {len(refinement_regions_df)}")
    print("\nCell Size Statistics:")
    print(refinement_regions_df['CellSize'].describe())
    
    # Group by Shape Type
    shape_type_counts = refinement_regions_df['ShapeType'].value_counts()
    print("\nRefinement Region Shape Types:")
    print(shape_type_counts)
    
    # Plot Shape Type distribution
    plt.figure(figsize=(10, 6))
    shape_type_counts.plot(kind='bar')
    plt.title('Distribution of Refinement Region Shape Types')
    plt.xlabel('Shape Type')
    plt.ylabel('Count')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

No refinement regions found in the geometry file.


In [57]:
# Extract Plan Parameters 
plan_parameters_df = HdfPlan.get_plan_parameters(plan_hdf_path)

2025-03-19 17:40:12 - ras_commander.HdfPlan - INFO - Using HDF file from direct string path: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.p06.hdf
2025-03-19 17:40:12 - ras_commander.HdfPlan - INFO - Final validated HDF file path: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.p06.hdf


In [58]:
plan_parameters_df

['   Plan                                   Parameter  \\\n', '0    06                                    1D Cores   \n', '1    06                           1D Flow Tolerance   \n', '2    06                       1D Maximum Iterations   \n', '3    06   1D Maximum Iterations Without Improvement   \n', '4    06     1D Maximum Water Surface Error To Abort   \n', '5    06                              1D Methodology   \n', '6    06         1D Storage Area Elevation Tolerance   \n', '7    06                                    1D Theta   \n', '8    06                             1D Theta Warmup   \n', '9    06        1D Water Surface Elevation Tolerance   \n', '10   06                        1D-2D Flow Tolerance   \n', '11   06  1D-2D Gate Flow Submergence Decay Exponent   \n', '12   06                    1D-2D IS Stablity Factor   \n', '13   06                    1D-2D LS Stablity Factor   \n', '14   06                    1D-2D Maximum Iterations   \n', '15   06         1D-2D Maximum Number 
...
[Output truncated, 7265 characters total]

In [59]:
# Extract volume accounting data
volume_accounting_df = HdfResultsPlan.get_volume_accounting(plan_hdf_path)

2025-03-19 17:40:12 - ras_commander.HdfResultsPlan - INFO - Using HDF file from direct string path: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.p06.hdf
2025-03-19 17:40:12 - ras_commander.HdfResultsPlan - INFO - Final validated HDF file path: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.p06.hdf


In [60]:
volume_accounting_df

['      Error  Error Percent  Precipitation Excess (acre feet)  \\\n', '0  0.469786       0.000332                       3756.872803   \n', '\n', '   Precipitation Excess (inches)  Total Boundary Flux of Water In  \\\n', '0                       1.707615                         141685.0   \n', '\n', '   Total Boundary Flux of Water Out Vol Accounting in  Volume Ending  \\\n', '0                      17240.263672         Acre Feet  124445.203125   \n', '\n', '   Volume Starting  \n', '0              0.0  ']

------

# RasPlanHdf Class Functions

-----

In [61]:
# Get plan start time as datetime object
start_time = HdfPlan.get_plan_start_time(plan_hdf_path)
print(f"Simulation start time: {start_time}")

2025-03-19 17:40:12 - ras_commander.HdfPlan - INFO - Using HDF file from direct string path: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.p06.hdf
2025-03-19 17:40:12 - ras_commander.HdfPlan - INFO - Final validated HDF file path: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.p06.hdf


Simulation start time: 2018-09-09 00:00:00


Simulation start time: 2018-09-09 00:00:00

In [62]:
# Get plan end time as datetime object
end_time = HdfPlan.get_plan_end_time(plan_hdf_path)
print(f"Simulation end time: {end_time}")

2025-03-19 17:40:12 - ras_commander.HdfPlan - INFO - Using HDF file from direct string path: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.p06.hdf
2025-03-19 17:40:12 - ras_commander.HdfPlan - INFO - Final validated HDF file path: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.p06.hdf


Simulation end time: 2018-09-14 00:00:00


Simulation end time: 2018-09-14 00:00:00

In [63]:
# Get maximum iteration count for mesh cells
max_iter_gdf = HdfResultsMesh.get_mesh_max_iter(plan_hdf_path)

2025-03-19 17:40:12 - ras_commander.HdfResultsMesh - INFO - Using HDF file from direct string path: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.p06.hdf
2025-03-19 17:40:12 - ras_commander.HdfResultsMesh - INFO - Final validated HDF file path: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.p06.hdf
2025-03-19 17:40:12 - ras_commander.HdfResultsMesh - INFO - Processing summary output for variable: Cell Last Iteration
2025-03-19 17:40:12 - ras_commander.HdfMesh - INFO - Using HDF file from h5py.File object: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.p06.hdf
2025-03-19 17:40:12 - ras_commander.HdfMesh - INFO - Final validated HDF file path: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.p06.hdf
2025-03-19 17:40:12 - ras_commander.HdfMesh - INFO - Using existing Path object HDF file: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2

In [64]:
max_iter_gdf

['         mesh_name  cell_id  cell_last_iteration  \\\n', '0      BaldEagleCr        0                    0   \n', '1      BaldEagleCr        1                    0   \n', '2      BaldEagleCr        2                    0   \n', '3      BaldEagleCr        3                    2   \n', '4      BaldEagleCr        4                    0   \n', '...            ...      ...                  ...   \n', '19592  BaldEagleCr    19592                    0   \n', '19593  BaldEagleCr    19593                    0   \n', '19594  BaldEagleCr    19594                    0   \n', '19595  BaldEagleCr    19595                    0   \n', '19596  BaldEagleCr    19596                    0   \n', '\n', '                             geometry  \n', '0              POINT (2083000 370750)  \n', '1              POINT (2083250 370750)  \n', '2              POINT (2083500 370750)  \n', '3              POINT (2083750 370750)  \n', '4              POINT (2084000 370750)  \n', '...                               ...  \n', '19592  POINT (1978423.032 300718.897)  \n', '19593  POINT (1973389.375 297311.928)  \n', '19594   POINT (1968834.79 295808.861)  \n', '19595  POINT (1966130.942 291879.395)  \n', '19596   POINT (1969660.046 289673.23)  \n', '\n', '[19597 rows x 4 columns]']

In [65]:
# Get cell coordinates 
cell_coords = HdfMesh.get_mesh_cell_points(plan_hdf_path)


2025-03-19 17:40:12 - ras_commander.HdfMesh - INFO - Using HDF file from direct string path: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.p06.hdf
2025-03-19 17:40:12 - ras_commander.HdfMesh - INFO - Final validated HDF file path: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.p06.hdf
2025-03-19 17:40:12 - ras_commander.HdfMesh - INFO - Using existing Path object HDF file: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.p06.hdf
2025-03-19 17:40:12 - ras_commander.HdfMesh - INFO - Final validated HDF file path: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.p06.hdf
2025-03-19 17:40:13 - ras_commander.HdfBase - INFO - Using HDF file from h5py.File object: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.p06.hdf
2025-03-19 17:40:13 - ras_commander.HdfBase - INFO - Final validated HDF file path: c:\GH\ras-commander\exam

In [66]:
# Plot Mesh Max Iterations

if generate_maps:
    # Extract x and y coordinates from the geometry column
    max_iter_gdf['x'] = max_iter_gdf['geometry'].apply(lambda geom: geom.x if geom is not None else None)
    max_iter_gdf['y'] = max_iter_gdf['geometry'].apply(lambda geom: geom.y if geom is not None else None)

    # Remove rows with None coordinates
    max_iter_gdf = max_iter_gdf.dropna(subset=['x', 'y'])

    # Create the plot
    fig, ax = plt.subplots(figsize=(12, 8))
    scatter = ax.scatter(max_iter_gdf['x'], max_iter_gdf['y'], 
                         c=max_iter_gdf['cell_last_iteration'], 
                         cmap='viridis', 
                         s=1)

    # Customize the plot
    ax.set_title('Max Iterations per Cell')
    ax.set_xlabel('X Coordinate')
    ax.set_ylabel('Y Coordinate')
    plt.colorbar(scatter, label='Max Iterations')

    # Show the plot
    plt.show()
else:
    print("generate_maps is set to False")

# Print the first few rows of the dataframe for verification
print("\nFirst few rows of the dataframe:")
max_iter_gdf[['mesh_name', 'cell_id', 'geometry']]



<Figure size 1200x800 with 2 Axes>


First few rows of the dataframe:


['         mesh_name  cell_id                        geometry\n', '0      BaldEagleCr        0          POINT (2083000 370750)\n', '1      BaldEagleCr        1          POINT (2083250 370750)\n', '2      BaldEagleCr        2          POINT (2083500 370750)\n', '3      BaldEagleCr        3          POINT (2083750 370750)\n', '4      BaldEagleCr        4          POINT (2084000 370750)\n', '...            ...      ...                             ...\n', '19592  BaldEagleCr    19592  POINT (1978423.032 300718.897)\n', '19593  BaldEagleCr    19593  POINT (1973389.375 297311.928)\n', '19594  BaldEagleCr    19594   POINT (1968834.79 295808.861)\n', '19595  BaldEagleCr    19595  POINT (1966130.942 291879.395)\n', '19596  BaldEagleCr    19596   POINT (1969660.046 289673.23)\n', '\n', '[19597 rows x 3 columns]']

In [67]:
# List top 10 points for Max Iteration per Cell
# Sort the dataframe by cell_last_iteration in descending order
top_iterations = max_iter_gdf.sort_values(by='cell_last_iteration', ascending=False).head(10)

# Create a more informative display with coordinates
print("\nTop 10 Cells with Highest Iteration Counts:")
top_iterations_display = top_iterations.copy()
top_iterations_display['x_coord'] = top_iterations_display['geometry'].apply(lambda geom: round(geom.x, 2))
top_iterations_display['y_coord'] = top_iterations_display['geometry'].apply(lambda geom: round(geom.y, 2))

# Display the results in a formatted table
print(top_iterations_display[['mesh_name', 'cell_id', 'cell_last_iteration', 'x_coord', 'y_coord']])



Top 10 Cells with Highest Iteration Counts:
         mesh_name  cell_id  cell_last_iteration     x_coord    y_coord
17721  BaldEagleCr    17721                 2652  2007386.39  321546.77
17752  BaldEagleCr    17752                  986  2005903.93  322873.01
17621  BaldEagleCr    17621                  509  1968250.00  292000.00
13520  BaldEagleCr    13520                  308  2007000.00  321250.00
13039  BaldEagleCr    13039                  194  2006750.00  324250.00
8749   BaldEagleCr     8749                  180  2028000.00  339000.00
16970  BaldEagleCr    16970                  148  1973750.00  296500.00
6415   BaldEagleCr     6415                  134  2054250.00  348500.00
6188   BaldEagleCr     6188                  133  2040250.00  349000.00
6692   BaldEagleCr     6692                  128  2041750.00  347500.00


In [68]:
# Get mesh maximum water surface elevation as Geodataframe
max_ws_gdf = HdfResultsMesh.get_mesh_max_ws(plan_hdf_path)

2025-03-19 17:40:13 - ras_commander.HdfResultsMesh - INFO - Using HDF file from direct string path: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.p06.hdf
2025-03-19 17:40:13 - ras_commander.HdfResultsMesh - INFO - Final validated HDF file path: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.p06.hdf
2025-03-19 17:40:13 - ras_commander.HdfResultsMesh - INFO - Processing summary output for variable: Maximum Water Surface
2025-03-19 17:40:13 - ras_commander.HdfMesh - INFO - Using HDF file from h5py.File object: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.p06.hdf
2025-03-19 17:40:13 - ras_commander.HdfMesh - INFO - Final validated HDF file path: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.p06.hdf
2025-03-19 17:40:13 - ras_commander.HdfMesh - INFO - Using existing Path object HDF file: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMult

In [69]:
# Check Dataframe Attributes (the HDF Attributes are also imported as Geoataframe Attributes)
max_ws_gdf.attrs

["{'mesh_name': 'BaldEagleCr',\n", " 'Max Time': np.float32(5.0),\n", " 'Max Value': np.float32(848.2054),\n", " 'Min Time': np.float32(0.99953705),\n", " 'Min Value': np.float32(535.585),\n", " 'Rows Variables': ['WSEL', 'Time'],\n", " 'Units': ['ft', 'days']}"]

In [70]:
max_ws_gdf

['         mesh_name  cell_id  maximum_water_surface maximum_water_surface_time  \\\n', '0      BaldEagleCr        0             704.054443        2018-09-10 18:00:00   \n', '1      BaldEagleCr        1             692.377991        2018-09-10 18:04:00   \n', '2      BaldEagleCr        2             671.183472        2018-09-10 18:13:20   \n', '3      BaldEagleCr        3             660.605469        2018-09-10 18:54:40   \n', '4      BaldEagleCr        4             660.586243        2018-09-10 18:55:20   \n', '...            ...      ...                    ...                        ...   \n', '19592  BaldEagleCr    19592               0.000000        2018-09-09 00:00:00   \n', '19593  BaldEagleCr    19593               0.000000        2018-09-09 00:00:00   \n', '19594  BaldEagleCr    19594               0.000000        2018-09-09 00:00:00   \n', '19595  BaldEagleCr    19595               0.000000        2018-09-09 00:00:00   \n', '19596  BaldEagleCr    19596               0.000000        2018-09-09 00:00:00   \n', '\n', '                             geometry  \n', '0              POINT (2083000 370750)  \n', '1              POINT (2083250 370750)  \n', '2              POINT (2083500 370750)  \n', '3              POINT (2083750 370750)  \n', '4              POINT (2084000 370750)  \n', '...                               ...  \n', '19592  POINT (1978423.032 300718.897)  \n', '19593  POINT (1973389.375 297311.928)  \n', '19594   POINT (1968834.79 295808.861)  \n', '19595  POINT (1966130.942 291879.395)  \n', '19596   POINT (1969660.046 289673.23)  \n', '\n', '[19597 rows x 5 columns]']

In [71]:
# Plot the max water surface as a map
if generate_maps:
    # Extract x and y coordinates from the geometry column
    max_ws_gdf['x'] = max_ws_gdf['geometry'].apply(lambda geom: geom.x if geom is not None else None)
    max_ws_gdf['y'] = max_ws_gdf['geometry'].apply(lambda geom: geom.y if geom is not None else None)

    # Remove rows with None coordinates
    max_ws_gdf = max_ws_gdf.dropna(subset=['x', 'y'])

    # Create the plot
    fig, ax = plt.subplots(figsize=(12, 8))
    scatter = ax.scatter(max_ws_gdf['x'], max_ws_gdf['y'], 
                         c=max_ws_gdf['maximum_water_surface'], 
                         cmap='viridis', 
                         s=10)

    # Customize the plot
    ax.set_title('Max Water Surface per Cell')
    ax.set_xlabel('X Coordinate')
    ax.set_ylabel('Y Coordinate')
    plt.colorbar(scatter, label='Max Water Surface (ft)')

    # Add grid lines
    ax.grid(True, linestyle='--', alpha=0.7)

    # Increase font size for better readability
    plt.rcParams.update({'font.size': 12})

    # Adjust layout to prevent cutting off labels
    plt.tight_layout()

    # Show the plot
    plt.show()
else:
    print("generate_maps is set to False")

<Figure size 1200x800 with 2 Axes>

In [72]:
# Plot the time of the max water surface elevation (WSEL)
if generate_maps:
    import matplotlib.dates as mdates
    from datetime import datetime

    # Convert the 'maximum_water_surface_time' to datetime objects
    max_ws_gdf['max_wsel_time'] = pd.to_datetime(max_ws_gdf['maximum_water_surface_time'])

    # Create the plot
    fig, ax = plt.subplots(figsize=(12, 8))

    # Convert datetime to hours since the start for colormap
    min_time = max_ws_gdf['max_wsel_time'].min()
    color_values = (max_ws_gdf['max_wsel_time'] - min_time).dt.total_seconds() / 3600  # Convert to hours

    scatter = ax.scatter(max_ws_gdf['x'], max_ws_gdf['y'], 
                        c=color_values, 
                        cmap='viridis', 
                        s=10)

    # Customize the plot
    ax.set_title('Time of Maximum Water Surface Elevation per Cell')
    ax.set_xlabel('X Coordinate')
    ax.set_ylabel('Y Coordinate')

    # Set up the colorbar
    cbar = plt.colorbar(scatter)
    cbar.set_label('Hours since simulation start')

    # Format the colorbar ticks to show hours
    cbar.set_ticks(range(0, int(color_values.max()) + 1, 6))  # Set ticks every 6 hours
    cbar.set_ticklabels([f'{h}h' for h in range(0, int(color_values.max()) + 1, 6)])

    # Add grid lines
    ax.grid(True, linestyle='--', alpha=0.7)

    # Increase font size for better readability
    plt.rcParams.update({'font.size': 12})

    # Adjust layout to prevent cutting off labels
    plt.tight_layout()

    # Show the plot
    plt.show()

    # Find the overall maximum WSEL and its time
    max_wsel_row = max_ws_gdf.loc[max_ws_gdf['maximum_water_surface'].idxmax()]
    hours_since_start = (max_wsel_row['max_wsel_time'] - min_time).total_seconds() / 3600
    print(f"\nOverall Maximum WSEL: {max_wsel_row['maximum_water_surface']:.2f} ft")
    print(f"Time of Overall Maximum WSEL: {max_wsel_row['max_wsel_time']}")
    print(f"Hours since simulation start: {hours_since_start:.2f} hours")
    print(f"Location of Overall Maximum WSEL: X={max_wsel_row['x']}, Y={max_wsel_row['y']}")


<Figure size 1200x800 with 2 Axes>


Overall Maximum WSEL: 848.21 ft
Time of Overall Maximum WSEL: 2018-09-10 17:00:00
Hours since simulation start: 41.00 hours
Location of Overall Maximum WSEL: X=1968500.0, Y=295000.0


In [73]:
# Get mesh minimum water surface elevation as geodataframe
min_ws_gdf = HdfResultsMesh.get_mesh_min_ws(plan_hdf_path)

2025-03-19 17:40:15 - ras_commander.HdfResultsMesh - INFO - Using HDF file from direct string path: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.p06.hdf
2025-03-19 17:40:15 - ras_commander.HdfResultsMesh - INFO - Final validated HDF file path: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.p06.hdf
2025-03-19 17:40:15 - ras_commander.HdfResultsMesh - INFO - Processing summary output for variable: Minimum Water Surface
2025-03-19 17:40:15 - ras_commander.HdfMesh - INFO - Using HDF file from h5py.File object: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.p06.hdf
2025-03-19 17:40:15 - ras_commander.HdfMesh - INFO - Final validated HDF file path: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.p06.hdf
2025-03-19 17:40:15 - ras_commander.HdfMesh - INFO - Using existing Path object HDF file: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMult

In [74]:
min_ws_gdf

['         mesh_name  cell_id  minimum_water_surface minimum_water_surface_time  \\\n', '0      BaldEagleCr        0             701.151245        2018-09-09 00:00:20   \n', '1      BaldEagleCr        1             689.052246        2018-09-09 00:00:20   \n', '2      BaldEagleCr        2             669.774719        2018-09-09 00:00:20   \n', '3      BaldEagleCr        3             658.986938        2018-09-09 00:00:20   \n', '4      BaldEagleCr        4             658.720581        2018-09-09 00:00:20   \n', '...            ...      ...                    ...                        ...   \n', '19592  BaldEagleCr    19592               0.000000        2018-09-09 00:00:00   \n', '19593  BaldEagleCr    19593               0.000000        2018-09-09 00:00:00   \n', '19594  BaldEagleCr    19594               0.000000        2018-09-09 00:00:00   \n', '19595  BaldEagleCr    19595               0.000000        2018-09-09 00:00:00   \n', '19596  BaldEagleCr    19596               0.000000        2018-09-09 00:00:00   \n', '\n', '                             geometry  \n', '0              POINT (2083000 370750)  \n', '1              POINT (2083250 370750)  \n', '2              POINT (2083500 370750)  \n', '3              POINT (2083750 370750)  \n', '4              POINT (2084000 370750)  \n', '...                               ...  \n', '19592  POINT (1978423.032 300718.897)  \n', '19593  POINT (1973389.375 297311.928)  \n', '19594   POINT (1968834.79 295808.861)  \n', '19595  POINT (1966130.942 291879.395)  \n', '19596   POINT (1969660.046 289673.23)  \n', '\n', '[19597 rows x 5 columns]']

In [75]:
# Get mesh maximum face velocity as geodataframe
max_face_v_gdf = HdfResultsMesh.get_mesh_max_face_v(plan_hdf_path)
print("\nMesh Max Face Velocity:")

2025-03-19 17:40:15 - ras_commander.HdfResultsMesh - INFO - Using HDF file from direct string path: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.p06.hdf
2025-03-19 17:40:15 - ras_commander.HdfResultsMesh - INFO - Final validated HDF file path: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.p06.hdf
2025-03-19 17:40:15 - ras_commander.HdfResultsMesh - INFO - Processing summary output for variable: Maximum Face Velocity
2025-03-19 17:40:16 - ras_commander.HdfMesh - INFO - Using HDF file from h5py.File object: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.p06.hdf
2025-03-19 17:40:16 - ras_commander.HdfMesh - INFO - Final validated HDF file path: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.p06.hdf
2025-03-19 17:40:16 - ras_commander.HdfMesh - INFO - Using existing Path object HDF file: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMult


Mesh Max Face Velocity:


In [76]:
max_face_v_gdf

['         mesh_name  face_id  maximum_face_velocity maximum_face_velocity_time  \\\n', '0      BaldEagleCr        0               0.451430        2018-09-10 18:00:00   \n', '1      BaldEagleCr        1              -0.887190        2018-09-10 18:00:40   \n', '2      BaldEagleCr        2              -0.498561        2018-09-10 18:00:40   \n', '3      BaldEagleCr        3              -0.304673        2018-09-10 18:00:40   \n', '4      BaldEagleCr        4              -0.762405        2018-09-10 10:00:40   \n', '...            ...      ...                    ...                        ...   \n', '37589  BaldEagleCr    37589               0.000000        2018-09-09 00:00:00   \n', '37590  BaldEagleCr    37590               0.000000        2018-09-09 00:00:00   \n', '37591  BaldEagleCr    37591               0.000000        2018-09-09 00:00:00   \n', '37592  BaldEagleCr    37592               0.000000        2018-09-09 00:00:00   \n', '37593  BaldEagleCr    37593               0.000000        2018-09-09 00:00:00   \n', '\n', '                                                geometry  \n', '0            LINESTRING (2042125 351625, 2042375 351625)  \n', '1            LINESTRING (2042375 351625, 2042375 351875)  \n', '2            LINESTRING (2042375 351875, 2042125 351875)  \n', '3            LINESTRING (2042125 351875, 2042125 351625)  \n', '4            LINESTRING (2042375 351375, 2042375 351625)  \n', '...                                                  ...  \n', '37589  LINESTRING (1978444.054 300812.794, 1978402.01...  \n', '37590  LINESTRING (1973531.76 297370.846, 1973246.989...  \n', '37591  LINESTRING (1968886.801 295900.5, 1968782.778 ...  \n', '37592  LINESTRING (1965981.775 291978.68, 1966280.11 ...  \n', '37593  LINESTRING (1969588.438 289578.114, 1969731.65...  \n', '\n', '[37594 rows x 5 columns]']

In [77]:
# Extract midpoint coordinates from the LineString geometries
max_face_v_gdf['x'] = max_face_v_gdf['geometry'].apply(lambda geom: geom.centroid.x)
max_face_v_gdf['y'] = max_face_v_gdf['geometry'].apply(lambda geom: geom.centroid.y)

# Create the plot
fig, ax = plt.subplots(figsize=(12, 8))
scatter = ax.scatter(max_face_v_gdf['x'], max_face_v_gdf['y'], 
                    c=max_face_v_gdf['maximum_face_velocity'].abs(),
                    cmap='viridis',
                    s=10)

# Customize the plot
ax.set_title('Max Face Velocity per Face')
ax.set_xlabel('X Coordinate') 
ax.set_ylabel('Y Coordinate')
plt.colorbar(scatter, label='Max Face Velocity (ft/s)')

# Add grid lines
ax.grid(True, linestyle='--', alpha=0.7)

# Increase font size for better readability
plt.rcParams.update({'font.size': 12})

# Adjust layout to prevent cutting off labels
plt.tight_layout()

# Show the plot
plt.show()

<Figure size 1200x800 with 2 Axes>

In [78]:
# Get mesh minimum face velocity as geodataframe
min_face_v_gdf = HdfResultsMesh.get_mesh_min_face_v(plan_hdf_path)


2025-03-19 17:40:18 - ras_commander.HdfResultsMesh - INFO - Using HDF file from direct string path: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.p06.hdf
2025-03-19 17:40:18 - ras_commander.HdfResultsMesh - INFO - Final validated HDF file path: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.p06.hdf
2025-03-19 17:40:18 - ras_commander.HdfResultsMesh - INFO - Processing summary output for variable: Minimum Face Velocity
2025-03-19 17:40:18 - ras_commander.HdfMesh - INFO - Using HDF file from h5py.File object: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.p06.hdf
2025-03-19 17:40:18 - ras_commander.HdfMesh - INFO - Final validated HDF file path: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.p06.hdf
2025-03-19 17:40:18 - ras_commander.HdfMesh - INFO - Using existing Path object HDF file: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMult

In [79]:
print("\nMesh Min Face Velocity:")
min_face_v_gdf


Mesh Min Face Velocity:


['         mesh_name  face_id  minimum_face_velocity minimum_face_velocity_time  \\\n', '0      BaldEagleCr        0                    0.0        2018-09-09 00:00:20   \n', '1      BaldEagleCr        1                    0.0        2018-09-09 00:00:20   \n', '2      BaldEagleCr        2                    0.0        2018-09-09 00:00:20   \n', '3      BaldEagleCr        3                    0.0        2018-09-09 00:00:20   \n', '4      BaldEagleCr        4                    0.0        2018-09-09 00:00:20   \n', '...            ...      ...                    ...                        ...   \n', '37589  BaldEagleCr    37589                    0.0        2018-09-09 00:00:20   \n', '37590  BaldEagleCr    37590                    0.0        2018-09-09 00:00:20   \n', '37591  BaldEagleCr    37591                    0.0        2018-09-09 00:00:20   \n', '37592  BaldEagleCr    37592                    0.0        2018-09-09 00:00:20   \n', '37593  BaldEagleCr    37593                    0.0        2018-09-09 00:00:20   \n', '\n', '                                                geometry  \n', '0            LINESTRING (2042125 351625, 2042375 351625)  \n', '1            LINESTRING (2042375 351625, 2042375 351875)  \n', '2            LINESTRING (2042375 351875, 2042125 351875)  \n', '3            LINESTRING (2042125 351875, 2042125 351625)  \n', '4            LINESTRING (2042375 351375, 2042375 351625)  \n', '...                                                  ...  \n', '37589  LINESTRING (1978444.054 300812.794, 1978402.01...  \n', '37590  LINESTRING (1973531.76 297370.846, 1973246.989...  \n', '37591  LINESTRING (1968886.801 295900.5, 1968782.778 ...  \n', '37592  LINESTRING (1965981.775 291978.68, 1966280.11 ...  \n', '37593  LINESTRING (1969588.438 289578.114, 1969731.65...  \n', '\n', '[37594 rows x 5 columns]']

In [80]:
# Get mesh max water surface error as geodataframe

max_ws_err_gdf = HdfResultsMesh.get_mesh_max_ws_err(plan_hdf_path)


2025-03-19 17:40:19 - ras_commander.HdfResultsMesh - INFO - Using HDF file from direct string path: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.p06.hdf
2025-03-19 17:40:19 - ras_commander.HdfResultsMesh - INFO - Final validated HDF file path: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.p06.hdf
2025-03-19 17:40:19 - ras_commander.HdfResultsMesh - INFO - Processing summary output for variable: Cell Maximum Water Surface Error
2025-03-19 17:40:19 - ras_commander.HdfMesh - INFO - Using HDF file from h5py.File object: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.p06.hdf
2025-03-19 17:40:19 - ras_commander.HdfMesh - INFO - Final validated HDF file path: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.p06.hdf
2025-03-19 17:40:19 - ras_commander.HdfMesh - INFO - Using existing Path object HDF file: c:\GH\ras-commander\examples\example_projects\BaldE

In [81]:
print("\nMesh Max Water Surface Error:")
max_ws_err_gdf



Mesh Max Water Surface Error:


['         mesh_name  cell_id  cell_maximum_water_surface_error  \\\n', '0      BaldEagleCr        0                          0.000206   \n', '1      BaldEagleCr        1                          0.001100   \n', '2      BaldEagleCr        2                          0.000795   \n', '3      BaldEagleCr        3                          0.000314   \n', '4      BaldEagleCr        4                          0.001795   \n', '...            ...      ...                               ...   \n', '19592  BaldEagleCr    19592                          0.000000   \n', '19593  BaldEagleCr    19593                          0.000000   \n', '19594  BaldEagleCr    19594                          0.000000   \n', '19595  BaldEagleCr    19595                          0.000000   \n', '19596  BaldEagleCr    19596                          0.000000   \n', '\n', '      cell_maximum_water_surface_error_time                        geometry  \n', '0                       2018-09-11 20:50:00          POINT (2083000 370750)  \n', '1                       2018-09-10 21:38:00          POINT (2083250 370750)  \n', '2                       2018-09-10 09:58:40          POINT (2083500 370750)  \n', '3                       2018-09-10 17:00:40          POINT (2083750 370750)  \n', '4                       2018-09-10 09:28:00          POINT (2084000 370750)  \n', '...                                     ...                             ...  \n', '19592                   2018-09-09 00:00:00  POINT (1978423.032 300718.897)  \n', '19593                   2018-09-09 00:00:00  POINT (1973389.375 297311.928)  \n', '19594                   2018-09-09 00:00:00   POINT (1968834.79 295808.861)  \n', '19595                   2018-09-09 00:00:00  POINT (1966130.942 291879.395)  \n', '19596                   2018-09-09 00:00:00   POINT (1969660.046 289673.23)  \n', '\n', '[19597 rows x 5 columns]']

In [82]:
# Plot max water surface error

if generate_maps:
# Extract x and y coordinates from the geometry points, handling None values
    max_ws_err_gdf['x'] = max_ws_err_gdf['geometry'].apply(lambda geom: geom.x if geom is not None else None)
    max_ws_err_gdf['y'] = max_ws_err_gdf['geometry'].apply(lambda geom: geom.y if geom is not None else None)

    # Remove any rows with None coordinates
    max_ws_err_gdf = max_ws_err_gdf.dropna(subset=['x', 'y'])

    # Create the plot
    fig, ax = plt.subplots(figsize=(12, 8))
    scatter = ax.scatter(max_ws_err_gdf['x'], max_ws_err_gdf['y'],
                        c=max_ws_err_gdf['cell_maximum_water_surface_error'],
                        cmap='viridis',
                        s=10)

    # Customize the plot
    ax.set_title('Max Water Surface Error per Cell')
    ax.set_xlabel('X Coordinate')
    ax.set_ylabel('Y Coordinate')
    plt.colorbar(scatter, label='Max Water Surface Error (ft)')

    # Add grid lines
    ax.grid(True, linestyle='--', alpha=0.7)

    # Increase font size for better readability
    plt.rcParams.update({'font.size': 12})

    # Adjust layout to prevent cutting off labels
    plt.tight_layout()

    # Show the plot
    plt.show()

<Figure size 1200x800 with 2 Axes>

In [83]:
# Sort Dataframe to show top 10 maximum water surface errors:
max_ws_err_gdf_sorted = max_ws_err_gdf.sort_values(by='cell_maximum_water_surface_error', ascending=False)


In [84]:
print("\nTop 10 maximum water surface errors:")
max_ws_err_gdf_sorted


Top 10 maximum water surface errors:


['         mesh_name  cell_id  cell_maximum_water_surface_error  \\\n', '1709   BaldEagleCr     1709                          0.009991   \n', '12789  BaldEagleCr    12789                          0.009981   \n', '5967   BaldEagleCr     5967                          0.009940   \n', '13627  BaldEagleCr    13627                          0.009938   \n', '1882   BaldEagleCr     1882                          0.009926   \n', '...            ...      ...                               ...   \n', '19489  BaldEagleCr    19489                          0.000000   \n', '19488  BaldEagleCr    19488                          0.000000   \n', '19487  BaldEagleCr    19487                          0.000000   \n', '19486  BaldEagleCr    19486                          0.000000   \n', '19568  BaldEagleCr    19568                          0.000000   \n', '\n', '      cell_maximum_water_surface_error_time                        geometry  \\\n', '1709                    2018-09-10 09:06:40          POINT (207725
...
[Output truncated, 2400 characters total]

In [85]:
# Get mesh summary output for other Datasets (here we retrieve Maximum Face Courant) as geodataframe
max_courant_gdf = HdfResultsMesh.get_mesh_summary(plan_hdf_path, var="Maximum Face Courant")

2025-03-19 17:40:20 - ras_commander.HdfResultsMesh - INFO - Using HDF file from direct string path: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.p06.hdf
2025-03-19 17:40:20 - ras_commander.HdfResultsMesh - INFO - Final validated HDF file path: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.p06.hdf
2025-03-19 17:40:20 - ras_commander.HdfResultsMesh - INFO - Processing summary output for variable: Maximum Face Courant
2025-03-19 17:40:20 - ras_commander.HdfMesh - INFO - Using HDF file from h5py.File object: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.p06.hdf
2025-03-19 17:40:20 - ras_commander.HdfMesh - INFO - Final validated HDF file path: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.p06.hdf
2025-03-19 17:40:20 - ras_commander.HdfMesh - INFO - Using existing Path object HDF file: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti

In [86]:
print("\nMesh Summary Output (Maximum Courant):")
max_courant_gdf.attrs


Mesh Summary Output (Maximum Courant):


["{'mesh_name': 'BaldEagleCr',\n", " 'Can Interpolate': 'False',\n", " 'Can Plot': 'True',\n", " 'Coverage': 'Average',\n", " 'Location': 'Faces',\n", " 'Maximum Value of Data Set': np.float32(0.6819884),\n", " 'Minimum Value of Data Set': np.float32(0.0),\n", " 'Name': 'Face Courant Maximum',\n", " 'Orientation': 'Scalar',\n", " 'Row': np.int32(0),\n", " 'Rows Variables': ['Courant Face', 'Time'],\n", " 'Units': 'vel*dt/length',\n", " 'Units per row': ['vel*dt/length', 'days']}"]

In [87]:
max_courant_gdf

['         mesh_name  face_id  maximum_face_courant maximum_face_courant_time  \\\n', '0      BaldEagleCr        0              0.000020       2018-09-09 11:03:20   \n', '1      BaldEagleCr        1              0.000062       2018-09-09 11:32:40   \n', '2      BaldEagleCr        2              0.000048       2018-09-10 07:02:40   \n', '3      BaldEagleCr        3              0.000016       2018-09-09 11:03:20   \n', '4      BaldEagleCr        4              0.000062       2018-09-09 11:05:20   \n', '...            ...      ...                   ...                       ...   \n', '37589  BaldEagleCr    37589              0.000000       2018-09-09 00:00:00   \n', '37590  BaldEagleCr    37590              0.000000       2018-09-09 00:00:00   \n', '37591  BaldEagleCr    37591              0.000000       2018-09-09 00:00:00   \n', '37592  BaldEagleCr    37592              0.000000       2018-09-09 00:00:00   \n', '37593  BaldEagleCr    37593              0.000000       2018-09-09 00:00:00   \n', '\n', '                                                geometry  \n', '0            LINESTRING (2042125 351625, 2042375 351625)  \n', '1            LINESTRING (2042375 351625, 2042375 351875)  \n', '2            LINESTRING (2042375 351875, 2042125 351875)  \n', '3            LINESTRING (2042125 351875, 2042125 351625)  \n', '4            LINESTRING (2042375 351375, 2042375 351625)  \n', '...                                                  ...  \n', '37589  LINESTRING (1978444.054 300812.794, 1978402.01...  \n', '37590  LINESTRING (1973531.76 297370.846, 1973246.989...  \n', '37591  LINESTRING (1968886.801 295900.5, 1968782.778 ...  \n', '37592  LINESTRING (1965981.775 291978.68, 1966280.11 ...  \n', '37593  LINESTRING (1969588.438 289578.114, 1969731.65...  \n', '\n', '[37594 rows x 5 columns]']

In [88]:
# Plot max Courant number

# Convert to GeoDataFrame if not empty
if not max_courant_gdf.empty:
    if generate_maps:
        # Get centroids of line geometries for plotting
        max_courant_gdf['centroid'] = max_courant_gdf.geometry.centroid
        max_courant_gdf['x'] = max_courant_gdf.centroid.x
        max_courant_gdf['y'] = max_courant_gdf.centroid.y

        # Create the plot
        fig, ax = plt.subplots(figsize=(12, 8))
        scatter = ax.scatter(max_courant_gdf['x'], max_courant_gdf['y'],
                        c=max_courant_gdf['maximum_face_courant'],
                        cmap='viridis',
                        s=10)

        # Customize the plot
        ax.set_title('Max Courant Number per Face')
        ax.set_xlabel('X Coordinate')
        ax.set_ylabel('Y Coordinate')
        plt.colorbar(scatter, label='Max Courant Number')

        # Add grid lines
        ax.grid(True, linestyle='--', alpha=0.7)

        # Increase font size for better readability
        plt.rcParams.update({'font.size': 12})

        # Adjust layout to prevent cutting off labels
        plt.tight_layout()

        # Show the plot
        plt.show()

# Print the first few rows of the dataframe for verification
print("\nFirst few rows of the Courant number dataframe:")
max_courant_gdf


<Figure size 1200x800 with 2 Axes>


First few rows of the Courant number dataframe:


['         mesh_name  face_id  maximum_face_courant maximum_face_courant_time  \\\n', '0      BaldEagleCr        0              0.000020       2018-09-09 11:03:20   \n', '1      BaldEagleCr        1              0.000062       2018-09-09 11:32:40   \n', '2      BaldEagleCr        2              0.000048       2018-09-10 07:02:40   \n', '3      BaldEagleCr        3              0.000016       2018-09-09 11:03:20   \n', '4      BaldEagleCr        4              0.000062       2018-09-09 11:05:20   \n', '...            ...      ...                   ...                       ...   \n', '37589  BaldEagleCr    37589              0.000000       2018-09-09 00:00:00   \n', '37590  BaldEagleCr    37590              0.000000       2018-09-09 00:00:00   \n', '37591  BaldEagleCr    37591              0.000000       2018-09-09 00:00:00   \n', '37592  BaldEagleCr    37592              0.000000       2018-09-09 00:00:00   \n', '37593  BaldEagleCr    37593              0.000000       2018-09-09 00:00:
...
[Output truncated, 2724 characters total]

In [89]:
# Get mesh summary output for other Datasets (here we retrieve Maximum Face Courant)

max_face_shear_gdf = HdfResultsMesh.get_mesh_summary(plan_hdf_path, var="Maximum Face Shear Stress")

2025-03-19 17:40:22 - ras_commander.HdfResultsMesh - INFO - Using HDF file from direct string path: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.p06.hdf
2025-03-19 17:40:22 - ras_commander.HdfResultsMesh - INFO - Final validated HDF file path: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.p06.hdf
2025-03-19 17:40:22 - ras_commander.HdfResultsMesh - INFO - Processing summary output for variable: Maximum Face Shear Stress
2025-03-19 17:40:22 - ras_commander.HdfMesh - INFO - Using HDF file from h5py.File object: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.p06.hdf
2025-03-19 17:40:22 - ras_commander.HdfMesh - INFO - Final validated HDF file path: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.p06.hdf
2025-03-19 17:40:22 - ras_commander.HdfMesh - INFO - Using existing Path object HDF file: c:\GH\ras-commander\examples\example_projects\BaldEagleCrk

In [90]:
print("\nMesh Summary Output (Maximum Face Shear Stress:")
print(max_face_shear_gdf.attrs)


Mesh Summary Output (Maximum Face Shear Stress:
{'mesh_name': 'BaldEagleCr', 'Max Time': np.float32(0.0), 'Max Value': np.float32(0.0), 'Min Time': np.float32(0.0), 'Min Value': np.float32(0.0), 'Rows Variables': ['Shear Stress', 'Time'], 'Units': ['PSF', 'days']}


In [91]:
max_face_shear_gdf

['         mesh_name  face_id  maximum_face_shear_stress  \\\n', '0      BaldEagleCr        0                        0.0   \n', '1      BaldEagleCr        1                        0.0   \n', '2      BaldEagleCr        2                        0.0   \n', '3      BaldEagleCr        3                        0.0   \n', '4      BaldEagleCr        4                        0.0   \n', '...            ...      ...                        ...   \n', '37589  BaldEagleCr    37589                        0.0   \n', '37590  BaldEagleCr    37590                        0.0   \n', '37591  BaldEagleCr    37591                        0.0   \n', '37592  BaldEagleCr    37592                        0.0   \n', '37593  BaldEagleCr    37593                        0.0   \n', '\n', '      maximum_face_shear_stress_time  \\\n', '0                         2018-09-09   \n', '1                         2018-09-09   \n', '2                         2018-09-09   \n', '3                         2018-09-09   \n', '4        
...
[Output truncated, 2112 characters total]

In [92]:
# Plot max face shear stress

if generate_maps and not max_face_shear_gdf.empty:
    # Calculate centroids of the line geometries and extract coordinates
    max_face_shear_gdf['centroid'] = max_face_shear_gdf['geometry'].apply(lambda line: line.centroid)
    max_face_shear_gdf['x'] = max_face_shear_gdf['centroid'].apply(lambda point: point.x)
    max_face_shear_gdf['y'] = max_face_shear_gdf['centroid'].apply(lambda point: point.y)

    # Create the plot
    fig, ax = plt.subplots(figsize=(12, 8))
    scatter = ax.scatter(max_face_shear_gdf['x'], max_face_shear_gdf['y'],
                        c=max_face_shear_gdf['maximum_face_shear_stress'],
                        cmap='viridis',
                        s=10)

    # Customize the plot
    ax.set_title('Max Face Shear Stress per Face')
    ax.set_xlabel('X Coordinate')
    ax.set_ylabel('Y Coordinate')
    plt.colorbar(scatter, label='Max Face Shear Stress (PSF)')

    # Add grid lines
    ax.grid(True, linestyle='--', alpha=0.7)

    # Increase font size for better readability
    plt.rcParams.update({'font.size': 12})

    # Adjust layout to prevent cutting off labels
    plt.tight_layout()

    # Show the plot
    plt.show()

<Figure size 1200x800 with 2 Axes>

In [93]:
# Get mesh summary output for Minimum Water Surface as geodataframe
summary_gdf_min_ws = HdfResultsMesh.get_mesh_summary(plan_hdf_path, var="Minimum Water Surface")

2025-03-19 17:40:25 - ras_commander.HdfResultsMesh - INFO - Using HDF file from direct string path: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.p06.hdf
2025-03-19 17:40:25 - ras_commander.HdfResultsMesh - INFO - Final validated HDF file path: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.p06.hdf
2025-03-19 17:40:25 - ras_commander.HdfResultsMesh - INFO - Processing summary output for variable: Minimum Water Surface
2025-03-19 17:40:25 - ras_commander.HdfMesh - INFO - Using HDF file from h5py.File object: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.p06.hdf
2025-03-19 17:40:25 - ras_commander.HdfMesh - INFO - Final validated HDF file path: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.p06.hdf
2025-03-19 17:40:25 - ras_commander.HdfMesh - INFO - Using existing Path object HDF file: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMult

In [94]:
print("\nMesh Summary Output (Minimum Water Surface):")
summary_gdf_min_ws


Mesh Summary Output (Minimum Water Surface):


['         mesh_name  cell_id  minimum_water_surface minimum_water_surface_time  \\\n', '0      BaldEagleCr        0             701.151245        2018-09-09 00:00:20   \n', '1      BaldEagleCr        1             689.052246        2018-09-09 00:00:20   \n', '2      BaldEagleCr        2             669.774719        2018-09-09 00:00:20   \n', '3      BaldEagleCr        3             658.986938        2018-09-09 00:00:20   \n', '4      BaldEagleCr        4             658.720581        2018-09-09 00:00:20   \n', '...            ...      ...                    ...                        ...   \n', '19592  BaldEagleCr    19592               0.000000        2018-09-09 00:00:00   \n', '19593  BaldEagleCr    19593               0.000000        2018-09-09 00:00:00   \n', '19594  BaldEagleCr    19594               0.000000        2018-09-09 00:00:00   \n', '19595  BaldEagleCr    19595               0.000000        2018-09-09 00:00:00   \n', '19596  BaldEagleCr    19596               0.000000        2018-09-09 00:00:00   \n', '\n', '                             geometry  \n', '0              POINT (2083000 370750)  \n', '1              POINT (2083250 370750)  \n', '2              POINT (2083500 370750)  \n', '3              POINT (2083750 370750)  \n', '4              POINT (2084000 370750)  \n', '...                               ...  \n', '19592  POINT (1978423.032 300718.897)  \n', '19593  POINT (1973389.375 297311.928)  \n', '19594   POINT (1968834.79 295808.861)  \n', '19595  POINT (1966130.942 291879.395)  \n', '19596   POINT (1969660.046 289673.23)  \n', '\n', '[19597 rows x 5 columns]']

In [95]:
# Get mesh summary output for Minimum Face Velocity as geodataframe
summary_gdf_min_fv = HdfResultsMesh.get_mesh_summary(plan_hdf_path, var="Minimum Face Velocity")

2025-03-19 17:40:25 - ras_commander.HdfResultsMesh - INFO - Using HDF file from direct string path: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.p06.hdf
2025-03-19 17:40:25 - ras_commander.HdfResultsMesh - INFO - Final validated HDF file path: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.p06.hdf
2025-03-19 17:40:25 - ras_commander.HdfResultsMesh - INFO - Processing summary output for variable: Minimum Face Velocity
2025-03-19 17:40:25 - ras_commander.HdfMesh - INFO - Using HDF file from h5py.File object: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.p06.hdf
2025-03-19 17:40:25 - ras_commander.HdfMesh - INFO - Final validated HDF file path: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.p06.hdf
2025-03-19 17:40:25 - ras_commander.HdfMesh - INFO - Using existing Path object HDF file: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMult

In [96]:
print("\nMesh Summary Output (Minimum Face Velocity):")
summary_gdf_min_fv


Mesh Summary Output (Minimum Face Velocity):


['         mesh_name  face_id  minimum_face_velocity minimum_face_velocity_time  \\\n', '0      BaldEagleCr        0                    0.0        2018-09-09 00:00:20   \n', '1      BaldEagleCr        1                    0.0        2018-09-09 00:00:20   \n', '2      BaldEagleCr        2                    0.0        2018-09-09 00:00:20   \n', '3      BaldEagleCr        3                    0.0        2018-09-09 00:00:20   \n', '4      BaldEagleCr        4                    0.0        2018-09-09 00:00:20   \n', '...            ...      ...                    ...                        ...   \n', '37589  BaldEagleCr    37589                    0.0        2018-09-09 00:00:20   \n', '37590  BaldEagleCr    37590                    0.0        2018-09-09 00:00:20   \n', '37591  BaldEagleCr    37591                    0.0        2018-09-09 00:00:20   \n', '37592  BaldEagleCr    37592                    0.0        2018-09-09 00:00:20   \n', '37593  BaldEagleCr    37593                    0.0        2018-09-09 00:00:20   \n', '\n', '                                                geometry  \n', '0            LINESTRING (2042125 351625, 2042375 351625)  \n', '1            LINESTRING (2042375 351625, 2042375 351875)  \n', '2            LINESTRING (2042375 351875, 2042125 351875)  \n', '3            LINESTRING (2042125 351875, 2042125 351625)  \n', '4            LINESTRING (2042375 351375, 2042375 351625)  \n', '...                                                  ...  \n', '37589  LINESTRING (1978444.054 300812.794, 1978402.01...  \n', '37590  LINESTRING (1973531.76 297370.846, 1973246.989...  \n', '37591  LINESTRING (1968886.801 295900.5, 1968782.778 ...  \n', '37592  LINESTRING (1965981.775 291978.68, 1966280.11 ...  \n', '37593  LINESTRING (1969588.438 289578.114, 1969731.65...  \n', '\n', '[37594 rows x 5 columns]']

In [97]:
# Get mesh summary output for Cell Cumulative Iteration as geodataframe
summary_gdf_cum_iter = HdfResultsMesh.get_mesh_summary(plan_hdf_path, var="Cell Cumulative Iteration")

2025-03-19 17:40:26 - ras_commander.HdfResultsMesh - INFO - Using HDF file from direct string path: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.p06.hdf
2025-03-19 17:40:26 - ras_commander.HdfResultsMesh - INFO - Final validated HDF file path: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.p06.hdf
2025-03-19 17:40:26 - ras_commander.HdfResultsMesh - INFO - Processing summary output for variable: Cell Cumulative Iteration
2025-03-19 17:40:26 - ras_commander.HdfMesh - INFO - Using HDF file from h5py.File object: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.p06.hdf
2025-03-19 17:40:26 - ras_commander.HdfMesh - INFO - Final validated HDF file path: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.p06.hdf
2025-03-19 17:40:26 - ras_commander.HdfMesh - INFO - Using existing Path object HDF file: c:\GH\ras-commander\examples\example_projects\BaldEagleCrk

In [98]:
print("\nMesh Summary Output (Cell Cumulative Iteration):")
summary_gdf_cum_iter


Mesh Summary Output (Cell Cumulative Iteration):


['         mesh_name  cell_id  cell_cumulative_iteration  \\\n', '0      BaldEagleCr        0                        0.0   \n', '1      BaldEagleCr        1                        0.0   \n', '2      BaldEagleCr        2                        0.0   \n', '3      BaldEagleCr        3                        0.0   \n', '4      BaldEagleCr        4                        0.0   \n', '...            ...      ...                        ...   \n', '19592  BaldEagleCr    19592                        0.0   \n', '19593  BaldEagleCr    19593                        0.0   \n', '19594  BaldEagleCr    19594                        0.0   \n', '19595  BaldEagleCr    19595                        0.0   \n', '19596  BaldEagleCr    19596                        0.0   \n', '\n', '                             geometry  \n', '0              POINT (2083000 370750)  \n', '1              POINT (2083250 370750)  \n', '2              POINT (2083500 370750)  \n', '3              POINT (2083750 370750)  \n', '4              POINT (2084000 370750)  \n', '...                               ...  \n', '19592  POINT (1978423.032 300718.897)  \n', '19593  POINT (1973389.375 297311.928)  \n', '19594   POINT (1968834.79 295808.861)  \n', '19595  POINT (1966130.942 291879.395)  \n', '19596   POINT (1969660.046 289673.23)  \n', '\n', '[19597 rows x 4 columns]']

In [99]:
# Get mesh timeseries output as xarray
# The mesh name is part of the timeseries HDF path, so you must pass the mesh_name to retrieve it

# Get mesh areas from previous code cell
mesh_areas = HdfMesh.get_mesh_area_names(geom_hdf_path)


2025-03-19 17:40:27 - ras_commander.HdfMesh - INFO - Using HDF file from direct string path: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.g09.hdf
2025-03-19 17:40:27 - ras_commander.HdfMesh - INFO - Final validated HDF file path: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.g09.hdf


In [100]:
mesh_areas

["['BaldEagleCr']"]

In [101]:
# Use the first mesh area name to extract mesh timeseries output as xarray
timeseries_xr = HdfResultsMesh.get_mesh_timeseries(plan_hdf_path, mesh_areas[0], "Water Surface") # Use the first 2D flow area name for mesh_name

2025-03-19 17:40:27 - ras_commander.HdfResultsMesh - INFO - Using HDF file from direct string path: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.p06.hdf
2025-03-19 17:40:27 - ras_commander.HdfResultsMesh - INFO - Final validated HDF file path: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.p06.hdf


In [102]:
timeseries_xr

xarray.DataArray with Unknown dimensions
[Full xarray output truncated during preprocessing]

In [103]:
# Time Series Output Variables for Cells
# 
# Variable Name: Description
# Water Surface: Water surface elevation
# Depth: Water depth
# Velocity: Magnitude of velocity
# Velocity X: X-component of velocity
# Velocity Y: Y-component of velocity
# Froude Number: Froude number
# Courant Number: Courant number
# Shear Stress: Shear stress on the bed
# Bed Elevation: Elevation of the bed
# Precipitation Rate: Rate of precipitation
# Infiltration Rate: Rate of infiltration
# Evaporation Rate: Rate of evaporation
# Percolation Rate: Rate of percolation
# Groundwater Elevation: Elevation of groundwater
# Groundwater Depth: Depth to groundwater
# Groundwater Flow: Groundwater flow rate
# Groundwater Velocity: Magnitude of groundwater velocity
# Groundwater Velocity X: X-component of groundwater velocity
# Groundwater Velocity Y: Y-component of groundwater velocity
# 
# These variables are available for time series output at the cell level in 2D flow areas.


In [104]:
# Get mesh cells timeseries output as xarray
cells_timeseries_xr = HdfResultsMesh.get_mesh_cells_timeseries(plan_hdf_path, mesh_areas[0])

2025-03-19 17:40:27 - ras_commander.HdfResultsMesh - INFO - Using HDF file from direct string path: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.p06.hdf
2025-03-19 17:40:28 - ras_commander.HdfResultsMesh - INFO - Final validated HDF file path: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.p06.hdf
2025-03-19 17:40:28 - ras_commander.HdfResultsMesh - WARNING - Variable 'Depth' not found in the HDF file for mesh 'BaldEagleCr'. Skipping.
2025-03-19 17:40:28 - ras_commander.HdfResultsMesh - WARNING - Variable 'Velocity' not found in the HDF file for mesh 'BaldEagleCr'. Skipping.
2025-03-19 17:40:28 - ras_commander.HdfResultsMesh - WARNING - Variable 'Velocity X' not found in the HDF file for mesh 'BaldEagleCr'. Skipping.
2025-03-19 17:40:28 - ras_commander.HdfResultsMesh - WARNING - Variable 'Velocity Y' not found in the HDF file for mesh 'BaldEagleCr'. Skipping.
2025-03-19 17:40:28 - ras_commander.HdfResultsMesh - WAR

In [105]:
cells_timeseries_xr

xarray.Dataset with Unknown dimensions
[Full xarray output truncated during preprocessing]

In [106]:
# Plot WSE Time Series Data (Random Cell ID) 
import matplotlib.pyplot as plt

if generate_plots:
    import numpy as np
    import random

    # Extract Water Surface data
    water_surface = cells_timeseries_xr[mesh_areas[0]]['Water Surface']

    # Get the time values
    time_values = water_surface.coords['time'].values

    # Pick a random cell_id
    random_cell_id = random.choice(water_surface.coords['cell_id'].values)

    # Extract the water surface elevation time series for the random cell
    wsel_timeseries = water_surface.sel(cell_id=random_cell_id)

    # Find the peak value and its index
    peak_value = wsel_timeseries.max().item()
    peak_index = wsel_timeseries.argmax().item()

    # Create the plot
    plt.figure(figsize=(12, 6))
    plt.plot(time_values, wsel_timeseries, label=f'Cell ID: {random_cell_id}')
    plt.scatter(time_values[peak_index], peak_value, color='red', s=100, zorder=5)
    plt.annotate(f'Peak: {peak_value:.2f} ft', 
                (time_values[peak_index], peak_value),
                xytext=(10, 10), textcoords='offset points',
                ha='left', va='bottom',
                bbox=dict(boxstyle='round,pad=0.5', fc='yellow', alpha=0.5),
                arrowprops=dict(arrowstyle='->', connectionstyle='arc3,rad=0'))

    plt.title(f'Water Surface Elevation Time Series for Random Cell (ID: {random_cell_id})')
    plt.xlabel('Time')
    plt.ylabel('Water Surface Elevation (ft)')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()

    # Log the plotting action
    logging.info(f"Plotted water surface elevation time series for random cell ID: {random_cell_id}")

    # Display the plot
    plt.show()

    # Print some statistics
    print(f"Statistics for Cell ID {random_cell_id}:")
    print(f"Minimum WSEL: {wsel_timeseries.min().item():.2f} ft")
    print(f"Maximum WSEL: {peak_value:.2f} ft")
    print(f"Mean WSEL: {wsel_timeseries.mean().item():.2f} ft")
    print(f"Time of peak: {time_values[peak_index]}")

2025-03-19 17:40:31 - root - INFO - Plotted water surface elevation time series for random cell ID: 13976


<Figure size 1200x600 with 1 Axes>

Statistics for Cell ID 13976:
Minimum WSEL: 674.85 ft
Maximum WSEL: 676.09 ft
Mean WSEL: 675.35 ft
Time of peak: 2018-09-10T18:00:00.000000000


In [107]:
# Get mesh faces timeseries output as xarray
faces_timeseries_xr = HdfResultsMesh.get_mesh_faces_timeseries(plan_hdf_path, mesh_areas[0])

2025-03-19 17:40:31 - ras_commander.HdfResultsMesh - INFO - Using HDF file from direct string path: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.p06.hdf
2025-03-19 17:40:31 - ras_commander.HdfResultsMesh - INFO - Final validated HDF file path: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.p06.hdf
2025-03-19 17:40:31 - ras_commander.HdfResultsMesh - INFO - Using existing Path object HDF file: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.p06.hdf
2025-03-19 17:40:31 - ras_commander.HdfResultsMesh - INFO - Final validated HDF file path: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.p06.hdf
2025-03-19 17:40:33 - ras_commander.HdfResultsMesh - INFO - Using existing Path object HDF file: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.p06.hdf
2025-03-19 17:40:33 - ras_commander.HdfResultsMesh - INFO - Final validate

In [108]:
faces_timeseries_xr

xarray.Dataset with Unknown dimensions
[Full xarray output truncated during preprocessing]

In [109]:
# Plot Random Face Results and Label Peak, Plus Map View

if generate_maps:

    # Select a random valid face ID number
    random_face = np.random.randint(0, faces_timeseries_xr.sizes['face_id'])

    # Extract time series data for the selected face
    variable = 'face_velocity'  # We could also use 'face_flow'
    face_data = faces_timeseries_xr[variable].sel(face_id=random_face)

    # Find peak value and its corresponding time
    peak_value = face_data.max().item()
    peak_time = face_data.idxmax().values

    # Plot time series
    plt.figure(figsize=(12, 8))
    plt.plot(faces_timeseries_xr.time, face_data)
    plt.title(f'{variable.capitalize()} Time Series for Face {random_face}')
    plt.xlabel('Time')
    plt.ylabel(f'{variable.capitalize()} ({faces_timeseries_xr.attrs["units"]})')
    plt.grid(True)

    # Annotate the peak point
    plt.annotate(f'Peak: ({peak_time}, {peak_value:.2f})', 
                (peak_time, peak_value),
                xytext=(10, 10), textcoords='offset points',
                arrowprops=dict(arrowstyle="->"))

    # Check for negative values and label the minimum if present
    min_value = face_data.min().item()
    if min_value < 0:
        min_time = face_data.idxmin().values
        plt.annotate(f'Min: ({min_time}, {min_value:.2f})', 
                    (min_time, min_value),
                    xytext=(10, -10), textcoords='offset points',
                    arrowprops=dict(arrowstyle="->"))

    plt.tight_layout()
    plt.show()

    # Create map view plot
    fig, ax = plt.subplots(figsize=(12, 8))


    # Calculate mesh faces extents with 10% buffer
    faces_bounds = mesh_cell_faces_gdf.total_bounds
    x_min, y_min, x_max, y_max = faces_bounds
    buffer_x = (x_max - x_min) * 0.1
    buffer_y = (y_max - y_min) * 0.1
    plot_xlim = [x_min - buffer_x, x_max + buffer_x]
    plot_ylim = [y_min - buffer_y, y_max + buffer_y]

    # Set plot limits before adding terrain
    ax.set_xlim(plot_xlim)
    ax.set_ylim(plot_ylim)

    # Add the terrain TIFF to the map, clipped to our desired extent
    tiff_path = Path.cwd() / 'example_projects' / 'BaldEagleCrkMulti2D' / 'Terrain' / 'Terrain50.baldeagledem.tif'
    with rasterio.open(tiff_path) as src:
        show(src, ax=ax, cmap='terrain', alpha=0.5)
        
    # Reset the limits after terrain plot
    ax.set_xlim(plot_xlim)
    ax.set_ylim(plot_ylim)

    # Plot all faces in gray
    mesh_cell_faces_gdf.plot(ax=ax, color='lightgray', alpha=0.5, zorder=2)

    # Get the selected face geometry
    selected_face = mesh_cell_faces_gdf[mesh_cell_faces_gdf['face_id'] == random_face]

    # Highlight the selected face in red
    selected_face.plot(
        ax=ax, 
        color='red',
        linewidth=2,
        label=f'Selected Face (ID: {random_face})',
        zorder=3
    )

    # Get bounds of selected face for zoomed inset
    bounds = selected_face.geometry.bounds.iloc[0]
    x_center = (bounds.iloc[0] + bounds.iloc[2]) / 2
    y_center = (bounds.iloc[1] + bounds.iloc[3]) / 2
    buffer = max(bounds.iloc[2] - bounds.iloc[0], bounds.iloc[3] - bounds.iloc[1]) * 2

    # Create zoomed inset with a larger size, inside the map frame
    axins = inset_axes(ax, width="70%", height="70%", loc='lower right',
                    bbox_to_anchor=(0.65, 0.05, 0.35, 0.35),
                    bbox_transform=ax.transAxes)

    # Plot terrain and faces in inset
    with rasterio.open(tiff_path) as src:
        show(src, ax=axins, cmap='terrain', alpha=0.5)
        
    # Plot zoomed view in inset
    mesh_cell_faces_gdf.plot(ax=axins, color='lightgray', alpha=0.5, zorder=2)
    selected_face.plot(ax=axins, color='red', linewidth=2, zorder=3)

    # Set inset limits with slightly more context
    axins.set_xlim(x_center - buffer/1.5, x_center + buffer/1.5)
    axins.set_ylim(y_center - buffer/1.5, y_center + buffer/1.5)

    # Remove inset ticks for cleaner look
    axins.set_xticks([])
    axins.set_yticks([])

    # Add a border to the inset
    for spine in axins.spines.values():
        spine.set_edgecolor('black')
        spine.set_linewidth(1.5)

    # Create connection lines between main plot and inset
    # Get the selected face centroid for connection point
    centroid = selected_face.geometry.centroid.iloc[0]
    con1 = ConnectionPatch(
        xyA=(centroid.x, centroid.y), coordsA=ax.transData,
        xyB=(0.02, 0.98), coordsB=axins.transAxes,
        arrowstyle="-", linestyle="--", color="gray", alpha=0.6
    )
    con2 = ConnectionPatch(
        xyA=(centroid.x, centroid.y), coordsA=ax.transData,
        xyB=(0.98, 0.02), coordsB=axins.transAxes,
        arrowstyle="-", linestyle="--", color="gray", alpha=0.6
    )

    ax.add_artist(con1)
    ax.add_artist(con2)

    # Add title and legend to main plot
    ax.set_title('Mesh Face Map View with Terrain')
    ax.legend()

    # Ensure equal aspect ratio while maintaining our desired extents
    ax.set_aspect('equal', adjustable='box')

    plt.tight_layout()
    plt.show()

    # Print summary information
    print(f"Random Face: {random_face}")
    print(f"Peak Value: {peak_value:.2f} {faces_timeseries_xr.attrs['units']} at {peak_time}")
    if min_value < 0:
        print(f"Minimum Value: {min_value:.2f} {faces_timeseries_xr.attrs['units']} at {min_time}")

    # Log the plotting action
    logging.info(f"Plotted mesh face time series and map view for random face ID: {random_face} with terrain")

<Figure size 1200x800 with 1 Axes>

C:\Users\billk\AppData\Local\Temp\ipykernel_2528\401875653.py:138: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()


<Figure size 1200x800 with 2 Axes>

2025-03-19 17:41:12 - root - INFO - Plotted mesh face time series and map view for random face ID: 10695 with terrain


Random Face: 10695
Peak Value: 1.18 ft/s at 2018-09-10T18:00:00.000000000


In [110]:
# Get meteorology precipitation attributes
meteo_precip_attrs = HdfPlan.get_plan_met_precip(plan_hdf_path)


2025-03-19 17:41:12 - ras_commander.HdfPlan - INFO - Using HDF file from direct string path: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.p06.hdf
2025-03-19 17:41:12 - ras_commander.HdfPlan - INFO - Final validated HDF file path: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.p06.hdf


In [111]:
meteo_precip_attrs

["{'DSS Filename': '.\\\\Precipitation\\\\precip.2018.09.dss',\n", " 'DSS Pathname': '/SHG/MARFC/PRECIP/01SEP2018:0200/01SEP2018:0300/NEXRAD/',\n", " 'Data Type': 'per-cum',\n", " 'Interpolation Method': '',\n", " 'Mode': 'Gridded',\n", ' \'Projection\': \'PROJCS["USA_Contiguous_Albers_Equal_Area_Conic_USGS_version",GEOGCS["NAD83",DATUM["North_American_Datum_1983",SPHEROID["GRS 1980",6378137,298.257222101,AUTHORITY["EPSG","7019"]],AUTHORITY["EPSG","6269"]],PRIMEM["Greenwich",0],UNIT["Degree",0.0174532925199433]],PROJECTION["Albers_Conic_Equal_Area"],PARAMETER["latitude_of_center",23],PARAMETER["longitude_of_center",-96],PARAMETER["standard_parallel_1",29.5],PARAMETER["standard_parallel_2",45.5],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]\',\n', " 'Raster Cellsize': np.float64(2000.0),\n", " 'Raster Cols': np.int32(515),\n", " 'Raster Left': np.float64(1096000.0),\n", " 'Raster Rows': np

In [112]:
# Get results unsteady attributes
results_unsteady_attrs = HdfResultsPlan.get_unsteady_info(plan_hdf_path)


2025-03-19 17:41:12 - ras_commander.HdfResultsPlan - INFO - Using HDF file from direct string path: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.p06.hdf
2025-03-19 17:41:12 - ras_commander.HdfResultsPlan - INFO - Final validated HDF file path: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.p06.hdf


In [113]:
results_unsteady_attrs

['                      Plan Title                     Program Name  \\\n', '0  Gridded Precip - Infiltration  HEC-RAS - River Analysis System   \n', '\n', '              Program Version  \\\n', '0  HEC-RAS 6.6 September 2024   \n', '\n', '                                   Project File Name  \\\n', '0  c:\\GH\\ras-commander\\examples\\example_projects\\...   \n', '\n', '                              Project Title                  Short ID  \\\n', '0  Bald Eagle Creek Example Dam Break Study  Grid Precip Infiltration   \n', '\n', '             Simulation Time Window             Type of Run  \n', '0  08Sep2018 2400 to 13Sep2018 2400  Unsteady Flow Analysis  ']

In [114]:
# Get results unsteady summary attributes
results_unsteady_summary_attrs = HdfResultsPlan.get_unsteady_summary(plan_hdf_path)


2025-03-19 17:41:12 - ras_commander.HdfResultsPlan - INFO - Using HDF file from direct string path: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.p06.hdf
2025-03-19 17:41:12 - ras_commander.HdfResultsPlan - INFO - Final validated HDF file path: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.p06.hdf


In [115]:
results_unsteady_summary_attrs

['  Computation Time DSS Computation Time Total  Maximum WSEL Error  \\\n', '0             00:00:00               00:04:25                 0.0   \n', '\n', '   Maximum number of cores                           Run Time Window  \\\n', '0                        4  19MAR2025 17:33:30 to 19MAR2025 17:37:54   \n', '\n', '                         Solution  Time Solution Went Unstable  \\\n', '0  Unsteady Finished Successfully                          NaN   \n', '\n', '  Time Stamp Solution Went Unstable  \n', '0                    Not Applicable  ']

In [116]:
# Get results volume accounting attributes
volume_accounting_attrs = HdfResultsPlan.get_volume_accounting(plan_hdf_path)

2025-03-19 17:41:12 - ras_commander.HdfResultsPlan - INFO - Using HDF file from direct string path: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.p06.hdf
2025-03-19 17:41:12 - ras_commander.HdfResultsPlan - INFO - Final validated HDF file path: c:\GH\ras-commander\examples\example_projects\BaldEagleCrkMulti2D\BaldEagleDamBrk.p06.hdf


In [117]:
volume_accounting_attrs

['      Error  Error Percent  Precipitation Excess (acre feet)  \\\n', '0  0.469786       0.000332                       3756.872803   \n', '\n', '   Precipitation Excess (inches)  Total Boundary Flux of Water In  \\\n', '0                       1.707615                         141685.0   \n', '\n', '   Total Boundary Flux of Water Out Vol Accounting in  Volume Ending  \\\n', '0                      17240.263672         Acre Feet  124445.203125   \n', '\n', '   Volume Starting  \n', '0              0.0  ']

-----

In [118]:
# Extract Compute Messages as String
print("Extracting Compute Messages")

import h5py
import numpy as np



def extract_string_from_hdf(results_hdf_filename: str, hdf_path: str) -> str:
    """
    Extract string from HDF object at a given path

    Parameters
    ----------
    results_hdf_filename : str
        Name of the HDF file
    hdf_path : str
        Path of the object in the HDF file

    Returns
    -------
    str
        Extracted string from the specified HDF object
    """
    with h5py.File(results_hdf_filename, 'r') as hdf_file:
        try:
            hdf_object = hdf_file[hdf_path]
            if isinstance(hdf_object, h5py.Group):
                return f"Group: {hdf_path}\nContents: {list(hdf_object.keys())}"
            elif isinstance(hdf_object, h5py.Dataset):
                data = hdf_object[()]
                if isinstance(data, bytes):
                    return data.decode('utf-8')
                elif isinstance(data, np.ndarray) and data.dtype.kind == 'S':
                    return [v.decode('utf-8') for v in data]
                else:
                    return str(data)
            else:
                return f"Unsupported object type: {type(hdf_object)}"
        except KeyError:
            return f"Path not found: {hdf_path}"



try:
    results_summary_string = extract_string_from_hdf(plan_hdf_path, '/Results/Summary/Compute Messages (text)')
    print("Compute Messages:")
    
    # Parse and print the compute messages in a more visually friendly way
    messages = results_summary_string[0].split('\r\n')
    
    for message in messages:
        if message.strip():  # Skip empty lines
            if ':' in message:
                key, value = message.split(':', 1)
                print(f"{key.strip():40} : {value.strip()}")
            else:
                print(f"\n{message.strip()}")
    
    # Print computation summary in a table format
    print("\nComputation Summary:")
    print("-" * 50)
    print(f"{'Computation Task':<30} {'Time':<20}")
    print("-" * 50)
    for line in messages:
        if 'Computation Task' in line:
            task, time = line.split('\t')
            print(f"{task:<30} {time:<20}")
    
    print("\nComputation Speed:")
    print("-" * 50)
    print(f"{'Task':<30} {'Simulation/Runtime':<20}")
    print("-" * 50)
    for line in messages:
        if 'Computation Speed' in line:
            task, speed = line.split('\t')
            print(f"{task:<30} {speed:<20}")

except Exception as e:
    print(f"Error extracting compute messages: {str(e)}")
    print("\nNote: If 'Results/Summary Output' is not in the file structure, it might indicate that the simulation didn't complete successfully or the results weren't saved properly.")

 



Extracting Compute Messages
Compute Messages:
Plan                                     : 'Gridded Precip - Infiltration' (BaldEagleDamBrk.p06)
Simulation started at                    : 19Mar2025 05:33:06 PM

Writing Plan GIS Data...

Completed Writing Plan GIS Data

Writing Geometry...
Computing 2D Flow Area 'BaldEagleCr' tables : Property tables do not exist.

2D Flow Area 'BaldEagleCr' tables complete 16.55 sec

Completed Writing Geometry

Writing Event Conditions ...

Processing Precipitation data...

(assumes geometry data is geo-referenced)

Finished Processing Precipitation data (2.291s)

Completed Writing Event Condition Data

Geometric Preprocessor HEC-RAS 6.6 September 2024

Finished Processing Geometry

Performing Unsteady Flow Simulation  HEC-RAS 6.6 September 2024
Unsteady Input Summary                   : 

2D Unsteady Diffusion Wave Equation Set (fastest)
2D number of Solver Cores                : 4

Maximum adaptive timestep = 40.0    Minimum adaptive timestep = 20.0

I

## Exploring HDF Datasets with HdfBase.get_dataset_info
This allows users to find HDF information that is not included in the ras-commander library.  Find the path in HDFView and set the group_path below to explore the HDF datasets and attributes.  Then, use the output to write your own function to extract the data.  

#### Get HDF Paths with Properties (For Exploring HDF Files)
HdfBase.get_dataset_info(plan_number, group_path="/")

For HDF datasets that are not supported by the RAS-Commadner library, provide the dataset path to HdfBase.get_dataset_info and provide the output to an LLM along with a relevent HDF* class(es) to generate new functions that extend the library's coverage.   